In [1]:
import pandas as pd
import datetime
pd.set_option('display.max_columns', None)
from pyexcelerate import Workbook
from decimal import Decimal

def yy(x,sp):
    return x[sp].sum()

def get_in_backup(path_backup,name):
    try:
        backup = pd.HDFStore(path_backup)
        ar  = backup[name]
        backup.close()
        return ar
    except:
        backup.close()

    
def save_pyex(path,df):
    wb = Workbook()
    wb.new_sheet('sheet1',data =  [df.columns.tolist(),]+df.values.tolist())
    wb.save(path)        
        
def rev_to_dict(ind,find):
#     ind = ind.astype('str')
    df = pd.DataFrame(index=ind,data=find.values,columns=[find.name])
    df = df.groupby(df.index).first()
    dictt =  df.to_dict('index')
    return dictt    

# впр    
def insert_vpr(x,dictt,name):
    try:
        return dictt[x][name]
    except KeyError:
        return 0
def save_backup(path_backup,arrs):
    try:
        backup = pd.HDFStore(path_backup)
        for arr in arrs:
            backup[str(arr.name)] = arr
        backup.close()   
    except:
        backup.close()    
        
import numpy as np
import time
from multiprocessing import Pool
import func_read_ex
def exlsx_f(df,year,month,ppm_date,nach_goda,sp_k,sp_kv,sp_summ,sp_kv_primech,sp_kv_sum_primech,sy,sp_current_kvartal):
    start_time = time.time()
    pool = Pool(processes=6)
    arrs =  np.array_split(df,18)
    arrs =  [(arr,year,month,ppm_date,nach_goda,sp_k,sp_kv,sp_summ,sp_kv_primech,sp_kv_sum_primech,sy,sp_current_kvartal) for arr in arrs]
    df_list = pool.map(func_read_ex.func_pr1, arrs)
    df = pd.concat(df_list)
    print("--- %s seconds ---" % (time.time() - start_time))
    pool.close()
    return df
        
def func_obesp_3month(x,year,month,sp):
    def yy(x,sp):
        return x[sp].sum()
    summ = yy(x,sp)
    if summ ==0:
        return x[str(month)+'/'+str(year)+' Остаток Кол-во']
    elif (summ -x[str(month)+'/'+str(year)+' Остаток Кол-во'])<=0:
        return summ
    else: 
        return x[str(month)+'/'+str(year)+' Остаток Кол-во']  
        
        
        
def func_obesp_3month_procentil(x,year,month):
    if x['ПП 3 месяца'] ==0:
        return ''
    elif (x[str(month)+'/'+str(year)+' Остаток Кол-во']/x['ПП 3 месяца'])>1:
        return 1
    else:
        return x[str(month)+'/'+str(year)+' Остаток Кол-во']/x['ПП 3 месяца']       
    
    
def func_obesp_first_month_procentil(x,year,month,sp):
    if yy(x,sp) ==0:
        return ''
    elif (x[str(month)+'/'+str(year)+' Остаток Кол-во']/yy(x,sp))>1:
        return 1
    else:
        return x[str(month)+'/'+str(year)+' Остаток Кол-во']/yy(x,sp)

def func_obesp_second_month_procentil(x,year,month,sp):
    if x['ПП кол-во '+str(month+2)+'/'+ str(year)] ==0:
        return ''
    elif x['ПП кол-во '+str(month+1)+'/'+ str(year)]>x[str(month)+'/'+str(year)+' Остаток Кол-во']:
        return 0
    elif ((x[str(month)+'/'+str(year)+' Остаток Кол-во']-yy(x,sp))/x['ПП кол-во '+str(month+2)+'/'+ str(year)])>1:
        return 1
    else:
        return ((x[str(month)+'/'+str(year)+' Остаток Кол-во']-yy(x,sp))/x['ПП кол-во '+str(month+2)+'/'+ str(year)])
    
def func_obesp_third_month_procentil(x,year,month,sp):
    if x['ПП кол-во '+str(month+3)+'/'+ str(year)] ==0:
        return ''
    elif (yy(x,sp)+ x['ПП кол-во '+str(month+2)+'/'+ str(year)])>x[str(month)+'/'+str(year)+' Остаток Кол-во']:
        return 0
    elif ((x[str(month)+'/'+str(year)+' Остаток Кол-во']-yy(x,sp)-x['ПП кол-во '+str(month+2)+'/'+ str(year)])/x['ПП кол-во '+str(month+3)+'/'+ str(year)])>1:
        return 1
    else:
        return ((x[str(month)+'/'+str(year)+' Остаток Кол-во']-yy(x,sp)-x['ПП кол-во '+str(month+2)+'/'+ str(year)])/x['ПП кол-во '+str(month+3)+'/'+ str(year)])

def func_ne_obesp_first_month_(x,year,month,sp):
    if (yy(x,sp)-x[str(month)+'/'+str(year)+' Остаток Кол-во'])>0:
        return yy(x,sp)-x[str(month)+'/'+str(year)+' Остаток Кол-во']
    else:
        return 0    

    
def func_ne_obesp_second_month_(x,year,month,sp):
    if (x['ПП кол-во '+str(month+2)+'/'+ str(year)]-x[str(month)+'/'+str(year)+' Остаток Кол-во']+yy(x,sp)-x['Не обеспечено запасом ' + str(month+1)+'/'+str(year)+ ' , кол-во'])>0:
        return x['ПП кол-во '+str(month+2)+'/'+ str(year)]-x[str(month)+'/'+str(year)+' Остаток Кол-во']+yy(x,sp)-x['Не обеспечено запасом ' + str(month+1)+'/'+str(year)+ ' , кол-во']
    else:
        return 0   
    
def func_ne_obesp_third_month_(x,year,month,sp):
    if (x['ПП кол-во '+str(month+2)+'/'+ str(year)]-x[str(month)+'/'+str(year)+' Остаток Кол-во']+yy(x,sp)+x['ПП кол-во '+str(month+3)+'/'+ str(year)]-x['Не обеспечено запасом ' + str(month+1)+'/'+str(year)+ ' , кол-во']-x['Не обеспечено запасом ' + str(month+2)+'/'+str(year)+ ' , кол-во'])>0:
        return x['ПП кол-во '+str(month+2)+'/'+ str(year)]-x[str(month)+'/'+str(year)+' Остаток Кол-во']+yy(x,sp)+x['ПП кол-во '+str(month+3)+'/'+ str(year)]-x['Не обеспечено запасом ' + str(month+1)+'/'+str(year)+ ' , кол-во']-x['Не обеспечено запасом ' + str(month+2)+'/'+str(year)+ ' , кол-во']
    else:
        return 0
    
def func_ZNB_AVIZOT_(x,year,month):
    summ = x['Не обеспечено запасом ' + str(month+1)+'/'+str(year)+ ' , кол-во']+x['Не обеспечено запасом ' + str(month+2)+'/'+str(year)+ ' , кол-во']+x['Не обеспечено запасом ' + str(month+3)+'/'+str(year)+ ' , кол-во']
    if summ==0:
        return 0
    elif (summ-x['Товар в пути (ZNB5, ZNB2/3-АвизОт)'])>=0:
        return x['Товар в пути (ZNB5, ZNB2/3-АвизОт)']
    elif (summ-x['Товар в пути (ZNB5, ZNB2/3-АвизОт)'])<0:
        return summ
    else:
        return 'хз'    
    
def func_EST_CONTRACT_(x,year,month):
    summ = x['Не обеспечено запасом ' + str(month+1)+'/'+str(year)+ ' , кол-во']+x['Не обеспечено запасом ' + str(month+2)+'/'+str(year)+ ' , кол-во']+x['Не обеспечено запасом ' + str(month+3)+'/'+str(year)+ ' , кол-во']
    if summ==0:
        return 0
    elif (summ-x['ZNB(АвизОт)']-x['Законтрактовано(ZNB-ПртПЗк)'])>=0:
        return x['Законтрактовано(ZNB-ПртПЗк)']
    elif (summ-x['ZNB(АвизОт)']-x['Законтрактовано(ZNB-ПртПЗк)'])<0:
        return summ-x['ZNB(АвизОт)']
    else:
        return 'хз'    
    
def func_NO_CONTRACT_(x,year,month):
    summ = x['Не обеспечено запасом ' + str(month+1)+'/'+str(year)+ ' , кол-во']+x['Не обеспечено запасом ' + str(month+2)+'/'+str(year)+ ' , кол-во']+x['Не обеспечено запасом ' + str(month+3)+'/'+str(year)+ ' , кол-во']
    if summ==0:
        return 0
    elif (summ-x['ZNB(АвизОт)']-x['Есть контракт']-x['Заявка на закупку(NB)'])>=0:
        return x['Заявка на закупку(NB)']
    elif (summ-x['ZNB(АвизОт)']-x['Есть контракт']-x['Заявка на закупку(NB)'])<0:
        return summ-x['ZNB(АвизОт)']-x['Есть контракт']
    else:
        return 'хз' 
    
months = {1:'Январь',2:'Февраль',3:'Март',4:'Апрель',5:'Май',6:'Июнь',7:'Июль',8:'Август',9:'Сентябрь',10:'Октябрь',11:'Ноябрь',12:'Декабрь',13:'Январь_с',14:'Февраль_с',15:'Март_с'}

In [2]:
# Когда начало года ставить True(есть только остаток на начало годе)
nach_goda = False
folder = '1120_curr'
# Для непосредственного формирования F0 
# current_date_f0 отчетный месяц -1 от ткущего (если сейчас февраль , то ставим январь)
current_date_f0 = datetime.datetime(2020,11,1)
first_month_of_kvartal = 10
last_month_of_kvartal = 12

# current_date = current_date_f0
ppm_date = datetime.datetime(2020,12,7)
# f0 =  get_in_backup(path_backup='backup_f0_cons.h5',name='c_f0_cons_11')
# был этот
# f0 =  get_in_backup(path_backup='f0_cons_11.h5',name='f0_cons_11')
# f0 =  get_in_backup(path_backup='backup_f0_cons.h5',name='c_f0_cons_01')
# df = get_in_backup(path_backup='backup_test_del.h5',name='extract')
f0 =  get_in_backup(path_backup='f0_cons_12_12032020_v2_for_obesp.h5',name='f0_cons_021')


df = get_in_backup(path_backup='backup_12.h5',name='extract')
directory = 'C:\\Users\\NKedrun\\Desktop\\kazzinc\\Uploads1\\'+folder+'\\'
iw13 = pd.read_excel(directory+'IW13.xlsx',sheet_name='Sheet1')
rcm = pd.read_excel(directory+'RCM.xlsx',sheet_name='Sheet1')
filter_else = get_in_backup(path_backup='backup_12.h5',name='filter_else')
if nach_goda:
    year = 2020
    month = 0
else:
    year = current_date_f0.year
    month = current_date_f0.month

print(year)
print(month)

2020
11


In [3]:

sp_k = []
sp_summ = []
sp_kv = []
sp_kv_sum = []
# # список месяцев потребности для формирования f0 ДЛЯ 1 месяца и считаем от остатков на начало года
# sp_k = ['ПП кол-во '+str(x)+'/'+str(current_date_f0.year) for x in  range(first_month_of_kvartal, current_date_f0.month+1)]
# sp_summ = ['ПП сумма '+str(x)+'/'+str(current_date_f0.year) for x in  range(first_month_of_kvartal, current_date_f0.month+1)]
sp_kv = ['ПП кол-во '+str(x)+'/'+str(current_date_f0.year) for x in  range(first_month_of_kvartal, current_date_f0.month+1)]
sp_kv_sum = ['ПП сумма '+str(x)+'/'+str(current_date_f0.year) for x in  range(first_month_of_kvartal, current_date_f0.month+1)]

sp_kv_primech = ['ПП кол-во '+str(x)+'/'+str(current_date_f0.year) for x in  range(first_month_of_kvartal, current_date_f0.month+2)]
sp_kv_sum_primech = ['ПП сумма '+str(x)+'/'+str(current_date_f0.year) for x in  range(first_month_of_kvartal, last_month_of_kvartal)]

sy = ['ПП кол-во '+str(x)+'/'+str(current_date_f0.year) for x in  range(first_month_of_kvartal, 13)]

sp_current_kvartal = ['ПП кол-во '+str(x)+'/'+str(current_date_f0.year) for x in  range(first_month_of_kvartal, last_month_of_kvartal+1)]

# список месяцев потребности для формирования f0 ДЛЯ 1 месяца и считаем от текущих остатков
if first_month_of_kvartal==1:
    sp_k = ['ПП кол-во '+str(x)+'/'+str(current_date_f0.year) for x in  range(first_month_of_kvartal, current_date_f0.month+4)]
    sp_summ = ['ПП сумма '+str(x)+'/'+str(current_date_f0.year) for x in  range(first_month_of_kvartal, current_date_f0.month+4)]
elif first_month_of_kvartal==4 or first_month_of_kvartal==7 or (first_month_of_kvartal==10 and current_date_f0.month ==10) :
    sp_k = ['ПП кол-во '+str(x)+'/'+str(current_date_f0.year) for x in  range(first_month_of_kvartal, current_date_f0.month+3)]
    sp_summ = ['ПП сумма '+str(x)+'/'+str(current_date_f0.year) for x in  range(first_month_of_kvartal, current_date_f0.month+3)]
elif first_month_of_kvartal==10 and current_date_f0.month ==11:
    sp_k = ['ПП кол-во '+str(x)+'/'+str(current_date_f0.year) for x in  range(first_month_of_kvartal, current_date_f0.month+2)]
    sp_k.append('ПП кол-во 1/'+str(current_date_f0.year +1))
    sp_summ = ['ПП сумма '+str(x)+'/'+str(current_date_f0.year) for x in  range(first_month_of_kvartal, current_date_f0.month+2)]
    sp_summ.append('ПП сумма 1/'+str(current_date_f0.year +1))
elif first_month_of_kvartal==10 and current_date_f0.month ==12:
    sp_k = ['ПП кол-во '+str(x)+'/'+str(current_date_f0.year) for x in  range(first_month_of_kvartal, current_date_f0.month+1)]
    sp_k.append('ПП кол-во 1/'+str(current_date_f0.year +1))
    sp_k.append('ПП кол-во 2/'+str(current_date_f0.year +1))
    sp_summ = ['ПП сумма '+str(x)+'/'+str(current_date_f0.year) for x in  range(first_month_of_kvartal, current_date_f0.month+1)]
    sp_summ.append('ПП сумма 1/'+str(current_date_f0.year +1))
    sp_summ.append('ПП сумма 2/'+str(current_date_f0.year +1))
    
    sp_kv_primech.append('ПП сумма 1/'+str(current_date_f0.year +1))
    sp_kv_sum_primech.append('ПП сумма 1/'+str(current_date_f0.year +1))
print(sp_k)
print(sp_summ)
print(sp_kv)
print(sp_kv_sum)
print(sp_kv_primech)
print(sp_kv_sum_primech)
print(sy)
print(sp_current_kvartal)

['ПП кол-во 10/2020', 'ПП кол-во 11/2020', 'ПП кол-во 12/2020', 'ПП кол-во 1/2021']
['ПП сумма 10/2020', 'ПП сумма 11/2020', 'ПП сумма 12/2020', 'ПП сумма 1/2021']
['ПП кол-во 10/2020', 'ПП кол-во 11/2020']
['ПП сумма 10/2020', 'ПП сумма 11/2020']
['ПП кол-во 10/2020', 'ПП кол-во 11/2020', 'ПП кол-во 12/2020']
['ПП сумма 10/2020', 'ПП сумма 11/2020']
['ПП кол-во 10/2020', 'ПП кол-во 11/2020', 'ПП кол-во 12/2020']
['ПП кол-во 10/2020', 'ПП кол-во 11/2020', 'ПП кол-во 12/2020']


In [4]:
# f0[str(1)+'/'+str(year)+' Остаток Кол-во']  = f0['Остаток на начало года Кол-во']
# f0[str(1)+'/'+str(year)+' Остаток Сумма']  = f0['Остаток на начало года Сумма']
f0.rename(columns={'2/2020 ФП Кол-во':'1/2020 ФП Кол-во','2/2020 ФП Сумма':'1/2020 ФП Сумма',\
                    '2/2020 ФC Кол-во':'1/2020 ФC Кол-во','2/2020 ФC Сумма':'1/2020 ФC Сумма',\
                    '2/2020 Остаток Кол-во':'1/2020 Остаток Кол-во','2/2020 Остаток Сумма':'1/2020 Остаток Сумма'},inplace = True)


In [5]:
# f0[str(1)+'/'+str(year)+' Остаток Кол-во']  = f0['Остаток на начало года Кол-во']
# f0[str(1)+'/'+str(year)+' Остаток Сумма']  = f0['Остаток на начало года Сумма']
f0.rename(columns={'3/2020 ФП Кол-во':'2/2020 ФП Кол-во','3/2020 ФП Сумма':'2/2020 ФП Сумма',\
                    '3/2020 ФC Кол-во':'2/2020 ФC Кол-во','3/2020 ФC Сумма':'2/2020 ФC Сумма',\
                    '3/2020 Остаток Кол-во':'2/2020 Остаток Кол-во','3/2020 Остаток Сумма':'2/2020 Остаток Сумма'},inplace = True)

In [6]:
f0.rename(columns={'4/2020 ФП Кол-во':'3/2020 ФП Кол-во','4/2020 ФП Сумма':'3/2020 ФП Сумма',\
                    '4/2020 ФC Кол-во':'3/2020 ФC Кол-во','4/2020 ФC Сумма':'3/2020 ФC Сумма',\
                    '4/2020 Остаток Кол-во':'3/2020 Остаток Кол-во','4/2020 Остаток Сумма':'3/2020 Остаток Сумма'},inplace = True)

In [7]:
f0.rename(columns={'5/2020 ФП Кол-во':'4/2020 ФП Кол-во','5/2020 ФП Сумма':'4/2020 ФП Сумма',\
                    '5/2020 ФC Кол-во':'4/2020 ФC Кол-во','5/2020 ФC Сумма':'4/2020 ФC Сумма',\
                    '5/2020 Остаток Кол-во':'4/2020 Остаток Кол-во','5/2020 Остаток Сумма':'4/2020 Остаток Сумма'},inplace = True)

In [8]:
f0.rename(columns={'6/2020 ФП Кол-во':'5/2020 ФП Кол-во','6/2020 ФП Сумма':'5/2020 ФП Сумма',\
                    '6/2020 ФC Кол-во':'5/2020 ФC Кол-во','6/2020 ФC Сумма':'5/2020 ФC Сумма',\
                    '6/2020 Остаток Кол-во':'5/2020 Остаток Кол-во','6/2020 Остаток Сумма':'5/2020 Остаток Сумма'},inplace = True)

In [9]:
f0.rename(columns={'7/2020 ФП Кол-во':'6/2020 ФП Кол-во','7/2020 ФП Сумма':'6/2020 ФП Сумма',\
                    '7/2020 ФC Кол-во':'6/2020 ФC Кол-во','7/2020 ФC Сумма':'6/2020 ФC Сумма',\
                    '7/2020 Остаток Кол-во':'6/2020 Остаток Кол-во','7/2020 Остаток Сумма':'6/2020 Остаток Сумма'},inplace = True)

In [10]:
f0.rename(columns={'8/2020 ФП Кол-во':'7/2020 ФП Кол-во','8/2020 ФП Сумма':'7/2020 ФП Сумма',\
                    '8/2020 ФC Кол-во':'7/2020 ФC Кол-во','8/2020 ФC Сумма':'7/2020 ФC Сумма',\
                    '8/2020 Остаток Кол-во':'7/2020 Остаток Кол-во','8/2020 Остаток Сумма':'7/2020 Остаток Сумма'},inplace = True)

In [11]:
f0.rename(columns={'9/2020 ФП Кол-во':'8/2020 ФП Кол-во','9/2020 ФП Сумма':'8/2020 ФП Сумма',\
                    '9/2020 ФC Кол-во':'8/2020 ФC Кол-во','9/2020 ФC Сумма':'8/2020 ФC Сумма',\
                    '9/2020 Остаток Кол-во':'8/2020 Остаток Кол-во','9/2020 Остаток Сумма':'8/2020 Остаток Сумма'},inplace = True)

In [12]:
f0.rename(columns={'10/2020 ФП Кол-во':'9/2020 ФП Кол-во','10/2020 ФП Сумма':'9/2020 ФП Сумма',\
                    '10/2020 ФC Кол-во':'9/2020 ФC Кол-во','10/2020 ФC Сумма':'9/2020 ФC Сумма',\
                    '10/2020 Остаток Кол-во':'9/2020 Остаток Кол-во','10/2020 Остаток Сумма':'9/2020 Остаток Сумма'},inplace = True)

In [13]:
f0.rename(columns={'11/2020 ФП Кол-во':'10/2020 ФП Кол-во','11/2020 ФП Сумма':'10/2020 ФП Сумма',\
                    '11/2020 ФC Кол-во':'10/2020 ФC Кол-во','11/2020 ФC Сумма':'10/2020 ФC Сумма',\
                    '11/2020 Остаток Кол-во':'10/2020 Остаток Кол-во','11/2020 Остаток Сумма':'10/2020 Остаток Сумма'},inplace = True)

In [14]:
f0.rename(columns={'12/2020 ФП Кол-во':'11/2020 ФП Кол-во','12/2020 ФП Сумма':'11/2020 ФП Сумма',\
                    '12/2020 ФC Кол-во':'11/2020 ФC Кол-во','12/2020 ФC Сумма':'11/2020 ФC Сумма',\
                    '12/2020 Остаток Кол-во':'11/2020 Остаток Кол-во','12/2020 Остаток Сумма':'12/2020 Остаток Сумма'},inplace = True)

In [16]:
# строка только для начала года
if nach_goda:
    f0.rename(columns={'Остаток на начало года Кол-во':str(month)+'/'+str(year)+' Остаток Кол-во',\
                  'Остаток на начало года Сумма':str(month)+'/'+str(year)+' Остаток Сумма'},inplace=True)

f0.reset_index(drop=True,inplace=True)
f0 = f0[(f0['Категория ТМЦ']=='Текущая потребность')&(f0.loc[:,[x for x in f0.columns if x.find('ПП кол-во')!=-1]].sum(axis = 1)>0)]
f0['СтатусМтрл на заводе'] = f0['СтатусМтрл на заводе'].apply(lambda x: 0 if x=='' else x).astype('int').astype('str')
f0['СтатусМтрл на заводе'] = f0['СтатусМтрл на заводе'].apply(lambda x: str(x).replace('.0','')).astype('int').astype('str')
if nach_goda:
    f0 = f0.drop(['Категория ТМЦ','БЕ','Имя Завода + Материал + Заказ','Завод + Склад','Завод + Область ППМ',\
                  'Код округа + Материал','Склад потребности','Область ППМ','Центральный завод','Код округа','Признак аналога',\
                  'Класс оценки','подгруппа','подподгруппа','Недропользование','ПлановСрокПоставки','SS (норма запаса на складе)'\
                  ,'SS сумма'],axis = 1).drop(list(f0.loc[:,[x for x in f0.columns if x.find('Остаток Кол-во')!=-1 and int(x.split('/')[0]) !=month ]]),axis = 1).drop(list(f0.loc[:,[x for x in f0.columns if x.find('Остаток Сумма')!=-1 and int(x.split('/')[0]) !=month ]]),axis = 1).drop([x for x in f0.columns if x.find('ФП')!=-1 or x.find('ФC')!=-1 or x.find('БЗ')!=-1 or x.find('Остаток прогноз')!=-1],axis = 1)
    
else:
        
    f0 = f0.drop(['Категория_запаса','Категория ТМЦ','БЕ','Склад потребности','Область ППМ','Центральный завод','Код округа','Признак аналога','Класс оценки','подгруппа','подподгруппа','Недропользование','ПлановСрокПоставки','SS (норма запаса на складе)','SS сумма','Кол-во запаса в начале периода','Сумма запаса в начале периода'],axis = 1).drop(list(f0.loc[:,[x for x in f0.columns if x.find('Остаток Кол-во')!=-1 and int(x.split('/')[0]) !=month ]]),axis = 1).drop(list(f0.loc[:,[x for x in f0.columns if x.find('Остаток Сумма')!=-1 and int(x.split('/')[0]) !=month ]]),axis = 1).drop([x for x in f0.columns if x.find('ФП')!=-1 or x.find('ФC')!=-1 or x.find('БЗ')!=-1 or x.find('Остаток прогноз')!=-1],axis = 1)

f0.loc[f0[f0['Заказ'].str.startswith('2')].index,'Заказ'] = '0'
aggregate= dict((x,'sum') for x in f0.columns if f0[x].dtype=='float64' and x!='Цена')
aggregate['Цена'] = ['max']
f0 = f0.groupby([x for x in f0.columns if f0[x].dtype=='object']).agg(aggregate).reset_index()
f0.columns =  [x[0] for x in f0.columns]
# f0[[x for x in f0.columns if f0[x].dtype=='float64']] = f0[[x for x in f0.columns if f0[x].dtype=='float64']].apply( lambda x: round(x,3))
f0['СтатусМтрл на заводе'] = f0['СтатусМтрл на заводе'].astype('int')

f0['Заказ+Материал'] = f0['Заказ']+' '+f0['Материал']
iw13['Заказ'] = iw13['Заказ'].astype('str')
spr_zak_mat_iw13 = rev_to_dict(iw13['Заказ']+' '+iw13['Материал'],iw13 ['Дата деблокирования'])
f0['Дата деблокирования (ТОРО)'] = f0['Заказ+Материал'].apply(lambda x:  insert_vpr(x,spr_zak_mat_iw13,'Дата деблокирования'))
f0.drop('Заказ+Материал',axis = 1,inplace = True)

f0['Завод+НН'] = f0['Завод'].astype('str')+' #'+f0['Материал'].astype('str')

spr_data_sozdanya_potr_df = rev_to_dict(df['Завод+НН'],df ['Дата создания потребности'])
f0['Дата создания потребности'] = f0['Завод+НН'].apply(lambda x:  insert_vpr(x,spr_data_sozdanya_potr_df,'Дата создания потребности'))
f0['Дата создания потребности'] = f0['Дата создания потребности'].apply(lambda x: '01.01.1900' if x =='00.00.0000' else  x)
f0['Дата создания потребности'] = f0['Дата создания потребности'].apply(lambda x: '01.01.1900' if x ==0  else  x)
f0['Дата создания потребности'] = [datetime.datetime(int(x.split('.')[2]),int(x.split('.')[1]),int(x.split('.')[0])) for x in f0['Дата создания потребности'].values]

spr_data_postaki_dlya_el_popolnreniya_ds = rev_to_dict(df['Завод+НН'],df ['Дата поставки для элемента пополнения'])
f0['Дата поставки для элемента пополнения'] = f0['Завод+НН'].apply(lambda x:  insert_vpr(x,spr_data_postaki_dlya_el_popolnreniya_ds,'Дата поставки для элемента пополнения'))
f0['Дата поставки для элемента пополнения'] = f0['Дата поставки для элемента пополнения'].apply(lambda x: '01.01.1900' if x =='00.00.0000' else  x)
f0['Дата поставки для элемента пополнения'] = f0['Дата поставки для элемента пополнения'].apply(lambda x: '01.01.1900' if x ==0  else  x)
f0['Дата поставки для элемента пополнения'] = [datetime.datetime(int(x.split('.')[2]),int(x.split('.')[1]),int(x.split('.')[0])) for x in f0['Дата поставки для элемента пополнения'].values]

f0['Заказчик + HH'] =f0['Комплекс'].astype('str')+' '+f0['Материал'].astype('str')


ZNB = pd.read_excel(directory+'ZNB.xlsx',sheet_name='Sheet1')
UB05 = pd.read_excel(directory+'UB05.xlsx',sheet_name='Sheet1')
spr_znb = pd.read_excel(directory+'Справодник для ZNB.xlsx',sheet_name='ZNB')
ZNB.fillna(0,inplace=True)
ZNB['Склад'] = ZNB['Склад'].apply(lambda x: '' if x==0 else x)
ZNB['Завод'] = ZNB['Завод'].apply(lambda x: str(x).replace('.0','')).astype('str')
ZNB['Завод+Склад'] = ZNB['Завод'].astype('str')+' '+ZNB['Склад'].astype('str')
spr_znb_ = rev_to_dict(spr_znb['Завод+склад'],spr_znb['Комплекс'])
ZNB['Комплекс']=ZNB['Завод+Склад'].apply(lambda x:  insert_vpr(x,spr_znb_,'Комплекс'))
ZNB['Комплекс+Маттериал'] = ZNB['Комплекс'].astype('str')+' '+ZNB['Материал'].astype('str')
ZNB['Завод+НН'] = ZNB['Завод'].astype('str')+' #'+ZNB['Материал'].astype('str')
ZNB.fillna(0,inplace=True)
ZNB['Карточка RCM'] = ZNB['Карточка RCM'].apply(lambda x: str(x).replace('.0','')).astype('str')




spr_kompl_mat_komp_znb_full = rev_to_dict(ZNB['Комплекс+Маттериал'],ZNB ['Карточка RCM'])
spr_mat_znb = rev_to_dict(ZNB['Материал'],ZNB ['Карточка RCM'])

for ind in f0.index:
    if (f0.at[ind,'Товар в пути (ZNB5, ZNB2/3-АвизОт)']   + f0.at[ind,'Законтрактовано(ZNB-ПртПЗк)'])>0:
        if insert_vpr(f0.at[ind,'Заказчик + HH'],spr_kompl_mat_komp_znb_full,'Карточка RCM')==0 and f0.at[ind,'Комплекс'] in  filter_else['one'].values:
            f0.at[ind,'RCM']  = insert_vpr(f0.at[ind,'Материал'],spr_mat_znb,'Карточка RCM')
        else:
            f0.at[ind,'RCM'] = insert_vpr(f0.at[ind,'Заказчик + HH'],spr_kompl_mat_komp_znb_full,'Карточка RCM')
    

spr_mat_kart_crm_inic_zak_znb = rev_to_dict(ZNB['Материал'].astype('str')+' '+ZNB ['Карточка RCM'], ZNB['Инициатор закупки']     )
f0['Материал+RCM'] = f0['Материал']+' '+f0['RCM'].astype('str')
f0['Трафик'] = f0['Материал+RCM'].apply(lambda x:  insert_vpr(x,spr_mat_kart_crm_inic_zak_znb,'Инициатор закупки'))
f0.drop('Материал+RCM',axis = 1,inplace = True)


rcm['Ид. случая'] = rcm['Ид. случая'].astype('str')
spr_dat_zak = rev_to_dict(rcm['Ид. случая'], rcm['Дата заключения'])
spr_dat_rcm_dat_okonch = rev_to_dict(rcm['Ид. случая'], rcm['Дата окончания действия'])
spr_dat_rcm_status = rev_to_dict(rcm['Ид. случая'], rcm['Статус названия'])
f0['Дата заключения RCM'] = f0['RCM'].apply(lambda x:  insert_vpr(x,spr_dat_zak,'Дата заключения'))
f0['Дата окончания RCM'] = f0['RCM'].apply(lambda x:  insert_vpr(x,spr_dat_rcm_dat_okonch,'Дата окончания действия'))
f0['Статус договора'] = f0['RCM'].apply(lambda x:  insert_vpr(x,spr_dat_rcm_status,'Статус названия'))

f0['Дата окончания RCM'].fillna(datetime.datetime(1900,1,1),inplace = True)
f0['Дата окончания RCM'] = f0['Дата окончания RCM'].apply(lambda x: datetime.datetime(1900,1,1) if x ==0  else x)
f0['Дата окончания RCM'] = f0['Дата окончания RCM'].apply(lambda x: 1900 if  x.year == 9999 else x)

f0['Дата заключения RCM'].fillna(datetime.datetime(1900,1,1),inplace = True)
f0['Дата заключения RCM'] = f0['Дата заключения RCM'].apply(lambda x: datetime.datetime(1900,1,1) if x ==0  else x)
f0['Дата заключения RCM'] = f0['Дата заключения RCM'].apply(lambda x: 1900 if  x.year == 9999 else x)

f0['Дата деблокирования (ТОРО)'].fillna(datetime.datetime(1900,1,1),inplace = True)
f0['Дата деблокирования (ТОРО)'] = f0['Дата деблокирования (ТОРО)'].apply(lambda x: datetime.datetime(1900,1,1) if x ==0  else x)
f0['Дата деблокирования (ТОРО)'] = f0['Дата деблокирования (ТОРО)'].apply(lambda x: 1900 if  x.year == 9999 else x)





NB = pd.read_excel(directory+'NB.xlsx',sheet_name='Sheet1')
NB = NB[~NB['Обозначение индикатора деблокирования'].isin(['Тендер завершен','блокиров.'])]
spr_nb = pd.read_excel(directory+'Справодник для NB.xlsx',sheet_name='NB')
NB.fillna(0,inplace = True)
spr_nb.fillna(0,inplace = True)
NB['Количество пополнения'] = NB['ЗатребованКолич']-NB['Заказанное к-во']
NB['Завод'] = NB['Завод'].apply(lambda x: str(x).replace('.0','')).astype('str')
NB['Склад'] = NB['Склад'].apply(lambda x: str(x).replace('.0','')).astype('str')
NB['Материал'] = NB['Материал'].apply(lambda x: str(x).replace('.0','')).astype('str')
NB['Материал'] = NB['Материал'].apply(lambda x: '0'+str(x) if  len(str(x))<9 else x)
NB['Склад'] = NB['Склад'].apply(lambda x: '' if x=='0' else x)
NB['Завод+Склад'] = NB['Завод']+' '+NB['Склад']
NB['Завод+Склад'] = NB['Завод+Склад'].str.strip()
spr_nb1 = rev_to_dict(spr_nb['Завод + Склад'].str.strip(),spr_nb['Комплекс'])
NB['Комплекс'] = NB['Завод+Склад'].apply(lambda x: insert_vpr(x,spr_nb1,'Комплекс'))
NB['Комплекс'] = NB['Комплекс'].apply(lambda x: str(x).replace('.0','')).astype('str')
NB['Комплекс+Маттериал'] = NB['Комплекс']+' '+NB['Материал']
NB['Количество пополнения'] = NB['Количество пополнения'].apply(lambda x:0 if x<0 else x)


spr_kompl_mat_komp_dat_zay = rev_to_dict(NB['Комплекс+Маттериал'],NB ['Дата заявки'])
spr_mat_dat_zay = rev_to_dict(NB['Материал'],NB ['Дата заявки'])

for ind in f0.index:
    if f0.at[ind,'Заявка на закупку(NB)']>0:
        if insert_vpr(f0.at[ind,'Заказчик + HH'],spr_kompl_mat_komp_dat_zay,'Дата заявки')==0 and f0.at[ind,'Комплекс'] in  filter_else['one'].values:
            if insert_vpr(f0.at[ind,'Материал'],spr_mat_dat_zay,'Дата заявки')==0:
                f0.at[ind,'Дата создания элемента ППМ пополнения NB01'] = datetime.datetime(1900,1,1)
            else:
                f0.at[ind,'Дата создания элемента ППМ пополнения NB01']  = insert_vpr(f0.at[ind,'Материал'],spr_mat_dat_zay,'Дата заявки')
        else:
            if insert_vpr(f0.at[ind,'Заказчик + HH'],spr_kompl_mat_komp_dat_zay,'Дата заявки')==0:
                f0.at[ind,'Дата создания элемента ППМ пополнения NB01'] = datetime.datetime(1900,1,1)
            else:
                f0.at[ind,'Дата создания элемента ППМ пополнения NB01'] = insert_vpr(f0.at[ind,'Заказчик + HH'],spr_kompl_mat_komp_dat_zay,'Дата заявки')
                
                

                
                
                
f0.fillna(0,inplace=True)               

C:\Users\NKedrun\Desktop\Portable Python 3.6.5 x64\App\Python\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\NKedrun\Desktop\Portable Python 3.6.5 x64\App\Python\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


In [17]:
f0['Дата создания элемента ППМ пополнения NB01'].unique()

array([Timestamp('2020-10-26 00:00:00'), Timestamp('2019-08-05 00:00:00'),
       0, Timestamp('2018-04-10 00:00:00'),
       Timestamp('2020-07-20 00:00:00'), Timestamp('2020-09-07 00:00:00'),
       Timestamp('2020-10-05 00:00:00'), Timestamp('2020-08-07 00:00:00'),
       Timestamp('2019-07-23 00:00:00'), Timestamp('2020-10-23 00:00:00'),
       Timestamp('2020-06-05 00:00:00'), Timestamp('2019-07-02 00:00:00'),
       Timestamp('2019-10-07 00:00:00'), Timestamp('2020-07-13 00:00:00'),
       Timestamp('2020-09-05 00:00:00'), Timestamp('2020-09-14 00:00:00'),
       Timestamp('2020-06-16 00:00:00'), Timestamp('2019-07-30 00:00:00'),
       Timestamp('2019-07-16 00:00:00'), Timestamp('2020-03-06 00:00:00'),
       Timestamp('2020-01-15 00:00:00'), Timestamp('2019-06-25 00:00:00'),
       Timestamp('2020-07-10 00:00:00'), Timestamp('2020-04-24 00:00:00'),
       Timestamp('2020-10-09 00:00:00'), Timestamp('2020-01-06 00:00:00'),
       Timestamp('2020-12-09 00:00:00'), Timestamp('2020

In [18]:
f0

,Модуль,Заказ,Материал,Комплекс,Завод,Имя Завода,КрТекстМатериала,Базовая ЕИ,СтатусМтрл на заводе,Плановик,группа,Завод+НН,ПРОЧЕЕ,ТОРО,РР,ПП кол-во 1/2020,ПП кол-во 2/2020,ПП кол-во 3/2020,ПП кол-во 4/2020,ПП кол-во 5/2020,ПП кол-во 6/2020,ПП кол-во 7/2020,ПП кол-во 8/2020,ПП кол-во 9/2020,ПП кол-во 10/2020,ПП кол-во 11/2020,ПП кол-во 12/2020,ПП кол-во 13/2020,ПП кол-во 14/2020,ПП кол-во 15/2020,Текущее кол-во ПП,ПП сумма 1/2020,ПП сумма 2/2020,ПП сумма 3/2020,ПП сумма 4/2020,ПП сумма 5/2020,ПП сумма 6/2020,ПП сумма 7/2020,ПП сумма 8/2020,ПП сумма 9/2020,ПП сумма 10/2020,ПП сумма 11/2020,ПП сумма 12/2020,ПП сумма 13/2020,ПП сумма 14/2020,ПП сумма 15/2020,Текущая сумма ПП,11/2020 Остаток Кол-во,"Товар в пути (ZNB5, ZNB2/3-АвизОт)",Заявка на закупку(NB),Законтрактовано(ZNB-ПртПЗк),Цена,Дата деблокирования (ТОРО),Дата создания потребности,Дата поставки для элемента пополнения,Заказчик + HH,RCM,Трафик,Дата заключения RCM,Дата окончания RCM,Статус договора,Дата создания элемента ППМ пополнения NB01
0,PP,0,010106022,РГОК,1303,Тишинский рудник РГОК,СПЕЦПРОФИЛЬ СВП-22,Т,0,Наяксова Е.А.,МЕТАЛЛОПРОКАТ,1303 #010106022,0,0,РР,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000,5.400,5.400,5.400,16.200,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000e+00,2.187000e+06,2.187000e+06,2.187000e+06,6.561000e+06,9.406,0.0,51.205,0.000,405000.00,1900-01-01,2020-10-22,2020-11-04,РГОК 010106022,0,0,1900-01-01,1900-01-01 00:00:00,0,2020-10-26 00:00:00
1,PP,0,010401016,УКМК,1202,Свинцовый завод УКМК,СТАЛЬ КРУГЛАЯ d16 СТ3ПС,Т,0,Наяксова Е.А.,МЕТАЛЛОПРОКАТ,1202 #010401016,ПРОЧЕЕ,0,РР,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.170,3.100,3.023,3.286,11.579,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.248688e+05,7.498125e+05,7.311881e+05,7.948012e+05,2.800671e+06,2.170,0.0,34.383,3.100,241875.00,1900-01-01,2020-11-30,2020-12-07,УКМК 010401016,300000052032,KSHIDERBAYEV,2020-02-17,2020-12-31 00:00:00,Действует,2019-08-05 00:00:00
2,PP,0,010409005,ГОК Алтай,1312,Малеевский рудник ЗГОК,"КАТАНКА d6,5 СТ3",КГ,0,Наяксова Е.А.,МЕТАЛЛОПРОКАТ,1312 #010409005,ПРОЧЕЕ,ТОРО,РР,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,424.293,398.618,435.275,410.617,1668.803,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.034214e+05,9.716314e+04,1.060983e+05,1.000879e+05,4.067707e+05,1668.803,0.0,0.000,0.000,243.75,1900-01-01,2020-12-01,2020-12-04,ГОК Алтай 010409005,0,0,1900-01-01,1900-01-01 00:00:00,0,0
3,PP,0,010409005,РГОК,1303,Тишинский рудник РГОК,"КАТАНКА d6,5 СТ3",КГ,0,Наяксова Е.А.,МЕТАЛЛОПРОКАТ,1303 #010409005,0,ТОРО,РР,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,70.733,0.000,0.000,0.000,70.733,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.724117e+04,0.000000e+00,0.000000e+00,0.000000e+00,1.724117e+04,70.733,0.0,0.000,0.000,243.75,1900-01-01,2020-11-27,2020-12-09,РГОК 010409005,0,0,1900-01-01,1900-01-01 00:00:00,0,0
4,PP,0,010801011,УКМК,1203,Медный завод УКМК,"ЛЕНТА СТАЛЬН 32х0,8ММ SPECTA FORTE MODEX",КГ,0,Наяксова Е.А.,МЕТАЛЛОПРОКАТ,1203 #010801011,0,0,РР,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6375.891,5856.265,5408.151,2875.464,20515.771,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.677874e+06,2.459631e+06,2.271423e+06,1.207695e+06,8.616624e+06,20560.541,0.0,0.000,345.234,420.00,1900-01-01,2020-11-30,2020-12-08,УКМК 010801011,300000043450,KSHIDERBAYEV,2019-10-21,2021-03-31 00:00:00,Действует,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
152439,ТОРО,101600001568,895002153,КАЗЦИНК-ТРАНС,1404,Казцинк-Транс Риддер,ЭЛЕМЕНТ ФИЛЬТРУЮЩИЙ 2655742471 / P173238,ШТ,0,Дворниченко Е.Н.,ЗАП.ЧАСТИ К ИМПОРТНОМУ САМОХОД НОМУ ОБОРУДОВАНИЮ,1404 #895002153,0,ТОРО,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.000,0.000,0.000,0.000,2.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.347000e+04,0.000000e+00,0.000000e+00,0.000000e+00,1.347000e+04,2.000,0.0,0.000,0.000,6735.00,1900-

In [19]:

# f0['ПП 3 месяца']  = f0.loc[:,sp_k].sum(axis=1)
# f0['ПП 3 месяца,Сумма']  = f0.loc[:,sp_summ].sum(axis=1)


# f0['Обеспечено 3 месяца, кол-во'] = f0.apply(lambda x:func_obesp_3month(x,year,month,sp_k),axis = 1 )
# f0['Обеспечено 3 месяца, сумма'] = f0['Обеспечено 3 месяца, кол-во']*f0['Цена']



# f0['Обеспеченность три месяца, кол-во, %'] = f0.apply(lambda x:func_obesp_3month_procentil(x,year,month,),axis = 1 )

# f0['Обеспеченность ' + str(month+1)+'/'+str(year)+ ' , кол-во, %'] = f0.apply(lambda x:func_obesp_first_month_procentil(x,year,month,sp_kv),axis = 1 )

# f0['Обеспеченность ' + str(month+2)+'/'+str(year)+ ' , кол-во, %'] = f0.apply(lambda x:func_obesp_second_month_procentil(x,year,month,sp_kv),axis = 1 )

# f0['Обеспеченность ' + str(month+3)+'/'+str(year)+ ' , кол-во, %'] = f0.apply(lambda x:func_obesp_third_month_procentil(x,year,month,sp_kv),axis = 1 )

# f0['Не обеспечено запасом ' + str(month+1)+'/'+str(year)+ ' , кол-во'] = f0.apply(lambda x:func_ne_obesp_first_month_(x,year,month,sp_kv),axis = 1 )

# f0['Не обеспечено запасом ' + str(month+2)+'/'+str(year)+ ' , кол-во'] = f0.apply(lambda x:func_ne_obesp_second_month_(x,year,month,sp_kv),axis = 1 )

# f0['Не обеспечено запасом ' + str(month+3)+'/'+str(year)+ ' , кол-во'] = f0.apply(lambda x:func_ne_obesp_third_month_(x,year,month,sp_kv),axis = 1 )

# f0['Не обеспечено, кол-во'] = f0.loc[:,[x for x in f0.columns if x.find('Не обеспечено запасом')!=-1]].sum(axis=1)  

# f0['Не обеспечено, сумма'] = f0['Не обеспечено, кол-во']* f0['Цена']

# f0['ZNB(АвизОт)'] = f0.apply(lambda x:func_ZNB_AVIZOT_(x,year,month),axis = 1 )

# f0['Есть контракт'] = f0.apply(lambda x:func_EST_CONTRACT_(x,year,month),axis = 1 )

# f0['Нет контракта'] = f0.apply(lambda x:func_NO_CONTRACT_(x,year,month),axis = 1 )

# arr_dec1 = [x for x in f0.columns if f0[x].dtype=='float64']

# for col in arr_dec1:
#     f0[col] = f0[col].apply(lambda x: round( Decimal(x),3))

sp_k=['ПП кол-во 11/2020', 'ПП кол-во 12/2020', 'ПП кол-во 13/2020']
sp_summ=['ПП сумма 11/2020', 'ПП сумма 12/2020', 'ПП сумма 13/2020']

f0 = exlsx_f(f0,year,month,ppm_date,nach_goda,sp_k,sp_kv,sp_summ,sp_kv_primech,sp_kv_sum_primech,sy,sp_current_kvartal)



--- 391.5409405231476 seconds ---


In [ ]:
sp_k

In [ ]:
f0[f0['Примечание 1']=='Обеспечено']

In [ ]:
# f0.name = 'obesp_no_f_01_t'
# save_backup('backup_obesp_t.h5',[f0])

In [20]:
f0.name = 'obesp_no_f_122020v2'
save_backup('backup_obesp.h5',[f0])

C:\Users\NKedrun\Desktop\Portable Python 3.6.5 x64\App\Python\lib\site-packages\ipykernel_launcher.py:2: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed-integer,key->block2_values] [items->Index(['Модуль', 'Заказ', 'Материал', 'Комплекс', 'Завод', 'Имя Завода',
       'КрТекстМатериала', 'Базовая ЕИ', 'Плановик', 'группа', 'Завод+НН',
       'ПРОЧЕЕ', 'ТОРО', 'РР', 'ПП кол-во 1/2020', 'ПП кол-во 2/2020',
       'ПП кол-во 3/2020', 'ПП кол-во 4/2020', 'ПП кол-во 5/2020',
       'ПП кол-во 6/2020', 'ПП кол-во 7/2020', 'ПП кол-во 8/2020',
       'ПП кол-во 9/2020', 'ПП кол-во 10/2020', 'ПП кол-во 11/2020',
       'ПП кол-во 12/2020', 'ПП кол-во 13/2020', 'ПП кол-во 14/2020',
       'ПП кол-во 15/2020', 'Текущее кол-во ПП', 'ПП сумма 1/2020',
       'ПП сумма 2/2020', 'ПП сумма 3/2020', 'ПП сумма 4/2020',
       'ПП сумма 5/2020', 'ПП сумма 6/2020', 'ПП сумма 7/2020',
       'ПП сумма 8/2020'

In [ ]:
f0 = get_in_backup(path_backup='backup_obesp.h5',name='obesp_no_f_032020v2')

In [21]:
for ind, i in enumerate(np.array_split(f0,2)) :
    save_pyex(df=i,path='C:\\Users\\NKedrun\\Desktop\\kazzinc\\Uploads1\\'+folder+'\\extr_xlsx\\obesp12_20_'+str(ind)+'.xlsx')

In [22]:
# формироваание итоговой таблицы
arr_potr = f0.loc[:,'ПП кол-во '+str(1)+'/'+ str(year):'ПП кол-во '+str(12)+'/'+ str(year)]
cols = arr_potr.columns
arr_potr.columns = ['Потребность '+ months[int(x.split('/')[0][-2:])] for x in cols]
f0.rename(columns={'Текущее кол-во ПП':'Потребность Итого','ПП 3 месяца':'Потребность 3 месяца, кол-во','ПП 3 месяца,Сумма':'Потребность 3 месяца, сумма',str(month)+'/'+str(year)+' Остаток Кол-во':'Остаток','Товар в пути (ZNB5, ZNB2/3-АвизОт)':'Товар в пути','Примечание 1':'Обеспеченность остатком','Примечание 2':'Период обеспеченности остатком','Примечание 3':'Обеспеченность документами пополнения'},inplace = True)


arr_potr1 = f0.loc[:,'Обеспеченность ' + str(month+1)+'/'+str(year)+ ' , кол-во, %':'Обеспеченность ' + str(month+3)+'/'+str(year)+ ' , кол-во, %']
cols1 = arr_potr1.columns
arr_potr1.columns = ['Обеспеченность '+ months[int(x.split('/')[0][-2:])]+' ,кол-во, %' for x in cols1]

arr_potr2 = f0.loc[:,'Не обеспечено запасом ' + str(month+1)+'/'+str(year)+ ' , кол-во':'Не обеспечено запасом ' + str(month+3)+'/'+str(year)+ ' , кол-во']
cols2 = arr_potr2.columns
arr_potr2.columns = ['Не обеспечено запасом '+ months[int(x.split('/')[0][-2:])]+' ,кол-во' for x in cols2]


final = pd.concat([f0['Завод'],f0['Имя Завода'],f0['Комплекс'],f0['Модуль'],f0['группа'],
                  f0['Материал'],f0['КрТекстМатериала'],f0['Базовая ЕИ'],f0['Заказ'],f0['Плановик'],f0['Трафик'],f0['Цена'],arr_potr,f0['Потребность Итого'],f0['Потребность 3 месяца, кол-во'],f0['Потребность 3 месяца, сумма'],f0['Остаток'],f0['Обеспечено 3 месяца, кол-во'],f0['Обеспечено 3 месяца, сумма'],f0['Обеспеченность три месяца, кол-во, %'],arr_potr1,arr_potr2,f0['Не обеспечено, кол-во'],f0['Не обеспечено, сумма'],f0['ZNB(АвизОт)'],f0['Есть контракт'],f0['Нет контракта'],f0['Заявка не принята в работу'],f0['Собственное производство'],f0['Излишек поставки'],f0['Товар в пути, сумма'],f0['Есть контракт, сумма'],f0['Нет контракта, сумма'],f0['Заявка не принята в работу, сумма'],f0['Собственное производство, сумма'],f0['Излишек поставки, сумма'],f0['Обеспеченность остатком'],f0['Период обеспеченности остатком'],f0['Обеспеченность документами пополнения'],f0['Расчетная дата потребности'],f0['Период потребности'],f0['Ввод потребности'],f0['Ответственность'],f0['Описание отклонения по ответственности'],f0['Комментарий по модулю'],f0['Мероприятия']],axis = 1)

# final = pd.concat([f0['Завод'],f0['Имя Завода'],f0['Комплекс'],f0['Модуль'],f0['группа'],
#                   f0['Материал'],f0['КрТекстМатериала'],f0['Базовая ЕИ'],f0['Заказ'],f0['Плановик'],f0['Трафик'],f0['Цена'],arr_potr,f0['Потребность Итого'],f0['Потребность 3 месяца, кол-во'],f0['Потребность 3 месяца, сумма'],f0['Остаток'],f0['Обеспечено 3 месяца, кол-во'],f0['Обеспечено 3 месяца, сумма'],f0['Обеспеченность три месяца, кол-во, %'],arr_potr1,arr_potr2,f0['Не обеспечено, кол-во'],f0['Не обеспечено, сумма'],f0['ZNB(АвизОт)'],f0['Есть контракт'],f0['Нет контракта'],f0['Заявка не принята в работу'],f0['Собственное производство'],f0['Излишек поставки'],f0['Товар в пути, сумма'],f0['Есть контракт, сумма'],f0['Нет контракта, сумма'],f0['Заявка не принята в работу, сумма'],f0['Собственное производство, сумма'],f0['Излишек поставки, сумма'],f0['Обеспеченность остатком'],f0['Период обеспеченности остатком'],f0['Обеспеченность документами пополнения'],f0['Расчетная дата потребности'],f0['Период потребности'],f0['Ввод потребности'],f0['Ответственность'],f0['Описание отклонения по ответственности'],f0['Комментарий по модулю'],f0['Комментарий по ПП прошедшего периода'],f0['Мероприятия'],f0['Потребность прошедшего периода, сумма']],axis = 1)

In [23]:
final.name = 'final_obesp_1220_v2'
save_backup('backup_obesp.h5',[final])

C:\Users\NKedrun\Desktop\Portable Python 3.6.5 x64\App\Python\lib\site-packages\ipykernel_launcher.py:2: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed-integer,key->block1_values] [items->Index(['Завод', 'Имя Завода', 'Комплекс', 'Модуль', 'группа', 'Материал',
       'КрТекстМатериала', 'Базовая ЕИ', 'Заказ', 'Плановик', 'Трафик', 'Цена',
       'Потребность Январь', 'Потребность Февраль', 'Потребность Март',
       'Потребность Апрель', 'Потребность Май', 'Потребность Июнь',
       'Потребность Июль', 'Потребность Август', 'Потребность Сентябрь',
       'Потребность Октябрь', 'Потребность Ноябрь', 'Потребность Декабрь',
       'Потребность Итого', 'Потребность 3 месяца, кол-во',
       'Потребность 3 месяца, сумма', 'Остаток', 'Обеспечено 3 месяца, кол-во',
       'Обеспечено 3 месяца, сумма', 'Обеспеченность три месяца, кол-во, %',
       'Обеспеченность Декабрь ,кол-во, %',
       'О

In [ ]:
final = get_in_backup(path_backup='backup_obesp.h5',name='final_obesp_0320_v2')

In [24]:
for ind, i in enumerate(np.array_split(final,2)) :
    save_pyex(df=i,path='C:\\Users\\NKedrun\\Desktop\\kazzinc\\Uploads1\\'+folder+'\\extr_xlsx\\obesp_final12_20_v2'+str(ind)+'.xlsx')

In [ ]:
import openpyxl
from openpyxl.reader.excel import load_workbook
from openpyxl.utils.dataframe import dataframe_to_rows
from openpyxl.styles import Font, Fill,PatternFill,Border,Side
from openpyxl.styles.colors import RGB
wb = load_workbook('C:\\Users\\Илья\\Desktop\\kazzinc\\Uploads1\\Образец_обеспеченность.xlsx',)

In [ ]:
final['Потребность 3 месяца, сумма'].sum()

In [ ]:
f0

In [ ]:

sheet = wb.get_sheet_by_name('ITOG')
rows = dataframe_to_rows(final[final['Комплекс']=='КДС'], header=True)
rows = [x[1] for x in enumerate(rows) if x[0]!=1 ]


border = Border(left=Side(color='a2a2d0',style='thin'),
                 right=Side(color='a2a2d0',style='thin'),
                top=Side(color='a2a2d0',style='thin'),
                bottom=Side(color='a2a2d0',style='thin')
)
no_fill = openpyxl.styles.PatternFill(fill_type=None)
side = openpyxl.styles.Side(border_style=None)
no_border = openpyxl.styles.borders.Border(
    left=side, 
    right=side, 
    top=side, 
    bottom=side,
)
# rd = sheet.row_dimensions[1].height = 100

# sheet.row_dimensions[1].chush = 50
for r_idx, row in enumerate(rows, 1):
    if r_idx>0:
        for c_idx, value in enumerate(row, 1):
            a = sheet.cell(row=r_idx, column=c_idx, value=value)
            a.fill = no_fill
            a.border = no_border
            if r_idx==1 and c_idx<=13:
                a.fill = PatternFill(start_color='dae0ef', end_color='dae0ef', fill_type='solid')
            if r_idx==1 and c_idx>=14 and  c_idx<=28: 
                a.fill = PatternFill(start_color='b5c0df', end_color='b5c0df', fill_type='solid')
            if r_idx==1 and c_idx>=29 and  c_idx<=53:   
                a.fill = PatternFill(start_color='bcd9de', end_color='bcd9de', fill_type='solid')
            if r_idx==1 and c_idx>=54 and  c_idx<=67:     
                a.fill = PatternFill(start_color='c4bbc6', end_color='c4bbc6', fill_type='solid')
            if   r_idx% 2!=0 and r_idx>1 :
                a.fill = PatternFill(start_color='dce0e8', end_color='dce0e8', fill_type='solid')
            a.border =   border  
# wb.save('C:\\Users\\Илья\\Desktop\\kazzinc\\Uploads1\\Образец_обеспеченность.xlsx')          



import win32com.client
office = win32com.client.Dispatch("Excel.Application")
wb = office.Workbooks.Open(r"C:\\Users\\Илья\\Desktop\\kazzinc\\Образец_обеспеченность.xlsx")
count = wb.Sheets.Count
for i in range(count):
    ws = wb.Worksheets[i]
    ws.Unprotect() # IF protected

    pivotCount = ws.PivotTables().Count
    for j in range(1, pivotCount+1):
        ws.PivotTables(j).SourceData = 'ITOG!R1C2:R'+str(len(rows))+'C63'
        ws.PivotTables(j).PivotCache().Refresh()
wb.Close(True)     

In [ ]:
import win32com.client
office = win32com.client.Dispatch("Excel.Application")
wb = office.Workbooks.Open(r"C:\\Users\\Илья\\Desktop\\kazzinc\\Образец_обеспеченность.xlsx")
sheet = wb.sheets('ITOG')
color = rgbToInt((220,224,232))
sheet.Range("A2:BL"+str(final.values.shape[0]+1)).Clear
sheet.Range("A2:BL"+str(final.values.shape[0]+1)).Borders.Color = rgbToInt((162,162,208))
for indR in range(2,final.values.shape[0]+2):
    for indC in range(2,final.values.shape[1]+2):
        
        if type(final.iloc[indR-2,indC-2]) is str:
            sheet.cells(indR,indC).NumberFormat = "@"
            sheet.cells(indR,indC).value = str(final.iloc[indR-2,indC-2])
        
        elif type(final.iloc[indR-2,indC-2]) is pd._libs.tslibs.timestamps.Timestamp:  
            sheet.cells(indR,indC).value = str(months[final.iloc[indR-2,indC-2].month])
        else:
            sheet.cells(indR,indC).NumberFormat = 0
            sheet.cells(indR,indC).value = int(final.iloc[indR-2,indC-2])
        sheet.cells(indR,1).value =indR-2 
        

        if   indR% 2!=0 and indR>1 :
            sheet.cells(indR,indC).Interior.Color  =color
        
count = wb.Sheets.Count
for i in range(count):
    ws = wb.Worksheets[i]
    ws.Unprotect() # IF protected

    pivotCount = ws.PivotTables().Count
    for j in range(1, pivotCount+1):
        ws.PivotTables(j).SourceData = 'ITOG!R1C2:R'+str(final.values.shape[0]+1)+'C63'
        ws.PivotTables(j).PivotCache().Refresh()            
wb.Close(True)           

In [ ]:
def rgbToInt(rgb):
    colorInt = rgb[0] + (rgb[1] * 256) + (rgb[2] * 256 * 256)
    return colorInt

In [ ]:
import win32com.client
win32c = win32com.client.constants
# office = win32com.client.Dispatch("Excel.Application")
office = win32com.client.gencache.EnsureDispatch("Excel.Application")
wb = office.Workbooks.Open(r"C:\\Users\\Илья\\Desktop\\kazzinc\\Образец_обеспеченность.xlsx")

In [ ]:
for ws in wb.Worksheets:
    for pvt in ws.PivotTables():
        if pvt.Name=='Сводная таблица2':
            ws.PivotTables(pvt.Name).AddDataField(ws.PivotTables(pvt.Name).PivotFields("Обеспеченность Сентябрь ,кол-во, %"),'Обеспеченность Сентябрь ,кол-во, %',win32c.xlAverage)
        ws.PivotTables(pvt.Name).PivotCache().Refresh()  
wb.Close(True,"C:\\Users\\Илья\\Desktop\\kazzinc\\Образец_обеспеченность1.xlsx")           

In [ ]:
get_in_backup(path_backup='backup_fine.h5',name='ost_nach_perioda')

In [ ]:
f0['Дата создания потребности'] = [datetime.datetime(int(x.split('.')[2]),int(x.split('.')[1]),int(x.split('.')[0])) for x in f0['Дата создания потребности'].values]

In [ ]:
f0['Дата создания потребности'] = f0['Дата создания потребности'].astype('str')

In [ ]:
f0['Дата создания потребности'] = [datetime.datetime(int(x.split('.')[2]),int(x.split('.')[1]),int(x.split('.')[0])) for x in f0['Дата создания потребности'].values if x!='0']

In [ ]:
f0.

In [ ]:
f0['Дата создания потребности'] = f0['Дата создания потребности'].apply(lambda x: '01.01.1900' if x =='00.00.0000' else  x)

In [ ]:
f0['Дата создания потребности'] = f0['Дата создания потребности'].astype('str')

In [ ]:
f0['Дата создания потребности'] = f0['Дата создания потребности'].apply(lambda x: '01.01.1900' if x =='00.00.0000'  else  x)
f0['Дата создания потребности'] = f0['Дата создания потребности'].apply(lambda x: '01.01.1900' if x ==0  else  x)

In [ ]:
f0.loc[:,[x for x in f0.columns if x.find('ПП кол-во')!=-1]].sum(axis = 1)

In [ ]:
f0[(f0['Категория ТМЦ']=='Текущая потребность')&(f0.loc[:,[x for x in f0.columns if x.find('ПП кол-во')!=-1]].sum(axis = 1)>0)]

In [ ]:
f0.shape

In [ ]:
for ind in f0.index:
        if f0.at[ind,'Примечание 1'] == 'Не обеспечено'  and f0.at[ind,'ПП кол-во '+str(current_date.month+1)+'/'+ str(current_date.year)]>0 and func_1(f0,ind,100)>(datetime.datetime(current_date.year,current_date.month+1,1)+datetime.timedelta(days=calendar.monthrange(current_date.year,current_date.month+1)[1])):
            f0.at[ind,'Ввод потребности'] = 'ПП введена без учета сроков контрактования'
        elif f0.at[ind,'Примечание 1'] == 'Не обеспечено'  and f0.at[ind,'ПП кол-во '+str(current_date.month+2)+'/'+ str(current_date.year)]>0 and func_1(f0,ind,100)>(datetime.datetime(current_date.year,current_date.month+2,1)+datetime.timedelta(days=calendar.monthrange(current_date.year,current_date.month+2)[1])):
            f0.at[ind,'Ввод потребности'] = 'ПП введена без учета сроков контрактования'
        elif f0.at[ind,'Примечание 1'] == 'Не обеспечено'  and f0.at[ind,'ПП кол-во '+str(current_date.month+3)+'/'+ str(current_date.year)]>0 and func_1(f0,ind,100)>(datetime.datetime(current_date.year,current_date.month+3,1)+datetime.timedelta(days=calendar.monthrange(current_date.year,current_date.month+3)[1])):
            f0.at[ind,'Ввод потребности'] = 'ПП введена без учета сроков контрактования'
        else:
            f0.at[ind,'Ввод потребности'] = 'ПП введена с учетом сроков контрактования'

In [ ]:
f0[(f0['Материал']=='031100010')&(f0['Завод+НН']=='1304 #031100010')]

In [ ]:
f0[(f0['Материал']=='031100010')&(f0['Завод+НН']=='1304 #031100010')]

In [ ]:
f0[f0['Материал']=='071004073']

In [ ]:
f0['Базовая ЕИ'].value_counts()

In [ ]:
ZNB[ZNB['Материал']=='895302083']

In [ ]:
f0[f0['Материал']=='895302083']

In [ ]:
spr_kompl_mat_komp_znb_full = rev_to_dict(ZNB['Комплекс+Маттериал'],ZNB ['Карточка RCM'])
spr_mat_znb = rev_to_dict(ZNB['Материал'],ZNB ['Карточка RCM'])

for ind in f0.index:
    if (f0.at[ind,'Товар в пути (ZNB5, ZNB2/3-АвизОт)']   + f0.at[ind,'Законтрактовано(ZNB-ПртПЗк)'])>0:
        if insert_vpr(f0.at[ind,'Заказчик + HH'],spr_kompl_mat_komp_znb_full,'Карточка RCM')==0 and f0.at[ind,'Комплекс'] in  filter_else['one'].values:
            f0.at[ind,'RCM']  = insert_vpr(f0.at[ind,'Материал'],spr_mat_znb,'Карточка RCM')
        else:
            f0.at[ind,'RCM'] = insert_vpr(f0.at[ind,'Заказчик + HH'],spr_kompl_mat_komp_znb_full,'Карточка RCM')

In [ ]:
spr_kompl_mat_komp_znb_full = rev_to_dict(ZNB['Комплекс+Маттериал'],ZNB ['Карточка RCM'])
spr_mat_znb = rev_to_dict(ZNB['Материал'],ZNB ['Карточка RCM'])

In [ ]:
spr_kompl_mat_komp_znb_full['ЖГОК 895302083']

In [ ]:
ZNB[ZNB['Комплекс+Маттериал']=='ЖГОК 895302083']

In [ ]:
ZNB[ZNB['Материал']=='895302083']

In [ ]:
spr_nb1 = rev_to_dict(spr_nb['Завод + Склад'].str.strip(),spr_nb['Комплекс'])

In [ ]:
spr_nb1

In [ ]:
f0[f0['Материал']=='895302083']

In [ ]:
f0

In [ ]:
f0.reset_index(drop=True,inplace=True)
f0 = f0[(f0['Категория ТМЦ']=='Текущая потребность')&(f0.loc[:,[x for x in f0.columns if x.find('ПП кол-во')!=-1]].sum(axis = 1)>0)]
f0['СтатусМтрл на заводе'] = f0['СтатусМтрл на заводе'].apply(lambda x: str(x).replace('.0','')).astype('int').astype('str')
f0 = f0.drop(['Категория ТМЦ','БЕ','Имя Завода + Материал + Заказ','Завод + Склад','Завод + Область ППМ','Код округа + Материал','Склад потребности','Область ППМ','Центральный завод','Код округа','Признак аналога','Класс оценки','подгруппа','подподгруппа','Недропользование','ПлановСрокПоставки','SS (норма запаса на складе)','SS сумма','Остаток на начало года Кол-во','Остаток на начало года Сумма'],axis = 1).drop(list(f0.loc[:,str(1)+'/'+str(current_date.year)+' Остаток Кол-во':str(current_date.month-1)+'/'+str(current_date.year)+' Остаток Кол-во'].columns),axis = 1).drop(list(f0.loc[:,str(1)+'/'+str(current_date.year)+' Остаток Сумма':str(current_date.month-1)+'/'+str(current_date.year)+' Остаток Сумма']),axis = 1).drop([x for x in f0.columns if x.find('ФП')!=-1 or x.find('ФC')!=-1 or x.find('БЗ')!=-1 or x.find('Остаток прогноз')!=-1],axis = 1)
f0.loc[f0[f0['Заказ'].str.startswith('2')].index,'Заказ'] = '0'
aggregate= dict((x,'sum') for x in f0.columns if f0[x].dtype=='float64' and x!='Цена')
aggregate['Цена'] = ['max']
f0 = f0.groupby([x for x in f0.columns if f0[x].dtype=='object']).agg(aggregate).reset_index()
f0.columns =  [x[0] for x in f0.columns]
# f0[[x for x in f0.columns if f0[x].dtype=='float64']] = f0[[x for x in f0.columns if f0[x].dtype=='float64']].apply( lambda x: round(x,3))
f0['СтатусМтрл на заводе'] = f0['СтатусМтрл на заводе'].astype('int')

In [ ]:
f0['ПП 3 месяца']  = f0.loc[:,'ПП кол-во '+str(current_date.month+1)+'/'+ str(current_date.year):'ПП кол-во '+str(current_date.month+3)+'/'+ str(current_date.year)].sum(axis=1)
f0['ПП 3 месяца,Сумма']  = f0.loc[:,'ПП сумма '+str(current_date.month+1)+'/'+ str(current_date.year):'ПП сумма '+str(current_date.month+3)+'/'+ str(current_date.year)].sum(axis=1)


f0['Обеспечено 3 месяца, кол-во'] = f0.apply(lambda x:func_obesp_3month(x,current_date),axis = 1 )
f0['Обеспечено 3 месяца, сумма'] = f0['Обеспечено 3 месяца, кол-во']*f0['Цена']



f0['Обеспеченность три месяца, кол-во, %'] = f0.apply(lambda x:func_obesp_3month_procentil(x,current_date),axis = 1 )

f0['Обеспеченность ' + str(current_date.month+1)+'/'+str(current_date.year)+ ' , кол-во, %'] = f0.apply(lambda x:func_obesp_first_month_procentil(x,current_date,sp_k),axis = 1 )

f0['Обеспеченность ' + str(current_date.month+2)+'/'+str(current_date.year)+ ' , кол-во, %'] = f0.apply(lambda x:func_obesp_second_month_procentil(x,current_date),axis = 1 )

f0['Обеспеченность ' + str(current_date.month+3)+'/'+str(current_date.year)+ ' , кол-во, %'] = f0.apply(lambda x:func_obesp_third_month_procentil(x,current_date),axis = 1 )

f0['Не обеспечено запасом ' + str(current_date.month+1)+'/'+str(current_date.year)+ ' , кол-во'] = f0.apply(lambda x:func_ne_obesp_first_month_(x,current_date),axis = 1 )

f0['Не обеспечено запасом ' + str(current_date.month+2)+'/'+str(current_date.year)+ ' , кол-во'] = f0.apply(lambda x:func_ne_obesp_second_month_(x,current_date),axis = 1 )

f0['Не обеспечено запасом ' + str(current_date.month+3)+'/'+str(current_date.year)+ ' , кол-во'] = f0.apply(lambda x:func_ne_obesp_third_month_(x,current_date),axis = 1 )

f0['Не обеспечено, кол-во'] = f0.loc[:,[x for x in f0.columns if x.find('Не обеспечено запасом')!=-1]].sum(axis=1)  

f0['Не обеспечено, сумма'] = f0['Не обеспечено, кол-во']* f0['Цена']

f0['ZNB(АвизОт)'] = f0.apply(lambda x:func_ZNB_AVIZOT_(x,current_date),axis = 1 )

f0['Есть контракт'] = f0.apply(lambda x:func_EST_CONTRACT_(x,current_date),axis = 1 )

f0['Нет контракта'] = f0.apply(lambda x:func_NO_CONTRACT_(x,current_date),axis = 1 )

arr_dec1 = [x for x in f0.columns if f0[x].dtype=='float64']

for col in arr_dec1:
    f0[col] = f0[col].apply(lambda x: round( Decimal(x),3))

In [ ]:
f99 = exlsx_f(f0,current_date,ppm_date)

In [ ]:
if current_date.month ==10:
    year1 = current_date.year
    year2 = current_date.year
    year3 = current_date.year+1
    
    month1 = current_date.month+1
    month2 = current_date.month+2
    month3 = 1
elif current_date.month == 11:
    year1 = current_date.year
    year2 = current_date.year+1
    year3 = current_date.year+1
    
    month1 = current_date.month+1
    month2 = 1
    month3 = 2
elif current_date.month==12:
    year1 = current_date.year+1
    year2 = current_date.year+1
    year3 = current_date.year+1
    
    month1 = 1
    month2 = 2
    month3 = 3
else:
    year1 = current_date.year
    year2 = current_date.year
    year3 = current_date.year
    month1 = current_date.month+1
    month2 = current_date.month+2
    month3 = current_date.month+3

for ind in f0.index:
        if f0.at[ind,'Ответственность']=='Ответственность Заказчика' and f0.at[ind,'Ввод потребности']=='ПП введена без учета сроков контрактования':
            f0.at[ind,'Описание отклонения по ответственности'] = 'Заявка введена без учета сроков поставки'
        elif f0.at[ind,'Примечание 1'] in ['Не обеспечено','Обеспечено частично'] and  f0.at[ind,'Плановик'] in ['Не закупать','Фиктивный плановик']:
            f0.at[ind,'Описание отклонения по ответственности'] = 'Некорректный плановик' 
        elif f0.at[ind,'Примечание 1'] in ['Не обеспечено','Обеспечено частично'] and f0.at[ind,'СтатусМтрл на заводе']>0 :
            f0.at[ind,'Описание отклонения по ответственности'] = 'Номенклатура заблокирована'    
        elif f0.at[ind,'Примечание 1'] in ['Не обеспечено','Обеспечено частично'] and f0.at[ind,'Плановик'] in ['Прд.РМБУК ППК КЦРС','КАЗЦИНКМАШ','Сбс.прв.ТООКазцинк','Прд.РМБ Ал ПК КЦРС','Прод.Отд.Диз.полиг','УММ','Собст.пр-во ЖГОК']:  
            f0.at[ind,'Описание отклонения по ответственности'] = 'Собственное производство'
        elif f0.at[ind,'Примечание 1'] in ['Не обеспечено','Обеспечено частично'] and (f0.at[ind,'Есть контракт']+f0.at[ind,'ZNB(АвизОт)'])>0 and   func_1(f0,ind,45)<f0.at[ind,'Дата заключения RCM']:    
            f0.at[ind,'Описание отклонения по ответственности'] = 'Контракт заключен позже необходимого срока'
        elif f0.at[ind,'Примечание 1'] in ['Не обеспечено','Обеспечено частично'] and (f0.at[ind,'Есть контракт']+f0.at[ind,'ZNB(АвизОт)'])==0 and f0.at[ind,'Нет контракта']==0 :      
            f0.at[ind,'Описание отклонения по ответственности'] = 'Заявки на закуп нет'
        elif f0.at[ind,'Примечание 1'] in ['Не обеспечено','Обеспечено частично'] and (f0.at[ind,'Есть контракт']+f0.at[ind,'ZNB(АвизОт)'])==0 and    f0.at[ind,'Дата создания элемента ППМ пополнения NB01']<=ppm_date:
            f0.at[ind,'Описание отклонения по ответственности'] = 'Контракт не заключен'
        elif f0.at[ind,'Примечание 1'] in ['Не обеспечено','Обеспечено частично'] and (f0.at[ind,'Есть контракт']+f0.at[ind,'ZNB(АвизОт)'])==0 and    f0.at[ind,'Дата создания элемента ППМ пополнения NB01']>=ppm_date:    
            f0.at[ind,'Описание отклонения по ответственности'] = 'Заявки на закуп сформированы в период между прогоном ППМ'
        elif f0.at[ind,'Примечание 1'] in ['Не обеспечено','Обеспечено частично'] and f0.at[ind,'ПП кол-во '+str(current_date.month+1)+'/'+ str(current_date.year)]>0 and  (f0.at[ind,'Есть контракт']+f0.at[ind,'ZNB(АвизОт)'])>0 and f0.at[ind,'ZNB(АвизОт)']>0 and (f0.at[ind,'Дата заключения RCM']+datetime.timedelta(days=65))<= (datetime.datetime(year1,month1,1)+datetime.timedelta(days=calendar.monthrange(year1,month1)[1])):    
            f0.at[ind,'Описание отклонения по ответственности'] = 'Заявка поставщику сформирована, срыв поставки поставщиком'
        elif f0.at[ind,'Примечание 1'] in ['Не обеспечено','Обеспечено частично'] and f0.at[ind,'ПП кол-во '+str(current_date.month+2)+'/'+ str(current_date.year)]>0 and  (f0.at[ind,'Есть контракт']+f0.at[ind,'ZNB(АвизОт)'])>0 and f0.at[ind,'ZNB(АвизОт)']>0 and (f0.at[ind,'Дата заключения RCM']+datetime.timedelta(days=65))<= (datetime.datetime(year2,month2,1)+datetime.timedelta(days=calendar.monthrange(year2,month2)[1])):     
            f0.at[ind,'Описание отклонения по ответственности'] = 'Заявка поставщику сформирована, срыв поставки поставщиком'
        elif f0.at[ind,'Примечание 1'] in ['Не обеспечено','Обеспечено частично'] and f0.at[ind,'ПП кол-во '+str(current_date.month+3)+'/'+ str(current_date.year)]>0 and  (f0.at[ind,'Есть контракт']+f0.at[ind,'ZNB(АвизОт)'])>0 and f0.at[ind,'ZNB(АвизОт)']>0 and (f0.at[ind,'Дата заключения RCM']+datetime.timedelta(days=65))<= (datetime.datetime(year3,month3,1)+datetime.timedelta(days=calendar.monthrange(year3,month3)[1])):
            f0.at[ind,'Описание отклонения по ответственности'] = 'Заявка поставщику сформирована, срыв поставки поставщиком'
        elif f0.at[ind,'Примечание 1'] in ['Не обеспечено','Обеспечено частично'] and f0.at[ind,'ПП кол-во '+str(current_date.month+1)+'/'+ str(current_date.year)]>0 and  (f0.at[ind,'Есть контракт']+f0.at[ind,'ZNB(АвизОт)'])>0 and f0.at[ind,'ZNB(АвизОт)']==0: 
            f0.at[ind,'Описание отклонения по ответственности'] = 'Заявка поставщику не сформирована'
        elif f0.at[ind,'Примечание 1'] in ['Не обеспечено','Обеспечено частично'] and f0.at[ind,'ПП кол-во '+str(current_date.month+2)+'/'+ str(current_date.year)]>0 and  (f0.at[ind,'Есть контракт']+f0.at[ind,'ZNB(АвизОт)'])>0 and f0.at[ind,'ZNB(АвизОт)']==0:     
            f0.at[ind,'Описание отклонения по ответственности'] = 'Заявка поставщику не сформирована'
        elif f0.at[ind,'Примечание 1'] in ['Не обеспечено','Обеспечено частично'] and f0.at[ind,'ПП кол-во '+str(current_date.month+3)+'/'+ str(current_date.year)]>0 and  (f0.at[ind,'Есть контракт']+f0.at[ind,'ZNB(АвизОт)'])>0 and f0.at[ind,'ZNB(АвизОт)']==0:     
            f0.at[ind,'Описание отклонения по ответственности'] = 'Заявка поставщику не сформирована'
        else:
            f0.at[ind,'Описание отклонения по ответственности'] = f0.at[ind,'Примечание 1']  

In [ ]:
for ind in f0.index:
        if f0.at[ind,'Примечание 1'] == 'Не обеспечено'  and f0.at[ind,'ПП кол-во '+str(current_date.month+1)+'/'+ str(current_date.year)]>0 and func_1(f0,ind,100)>(datetime.datetime(year1,month1,1)+datetime.timedelta(days=calendar.monthrange(year1,month1)[1])):
            f0.at[ind,'Ввод потребности'] = 'ПП введена без учета сроков контрактования'
        elif f0.at[ind,'Примечание 1'] == 'Не обеспечено'  and f0.at[ind,'ПП кол-во '+str(current_date.month+2)+'/'+ str(current_date.year)]>0 and func_1(f0,ind,100)>(datetime.datetime(year2,month2,1)+datetime.timedelta(days=calendar.monthrange(year2,month2)[1])):
            f0.at[ind,'Ввод потребности'] = 'ПП введена без учета сроков контрактования'
        elif f0.at[ind,'Примечание 1'] == 'Не обеспечено'  and f0.at[ind,'ПП кол-во '+str(current_date.month+3)+'/'+ str(current_date.year)]>0 and func_1(f0,ind,100)>(datetime.datetime(year3,month3,1)+datetime.timedelta(days=calendar.monthrange(year3,month3)[1])):
            f0.at[ind,'Ввод потребности'] = 'ПП введена без учета сроков контрактования'
        else:
            f0.at[ind,'Ввод потребности'] = 'ПП введена с учетом сроков контрактования'

In [ ]:
f0 =  get_in_backup(path_backup='backup_f0_cons.h5',name='c_f0_cons_08')

In [ ]:
f0 =  get_in_backup(path_backup='backup_obesp.h5',name='obesp_no_f_10')

In [ ]:
f0

In [ ]:
f0 = f0.drop(['Категория ТМЦ','БЕ','Имя Завода + Материал + Заказ','Завод + Склад','Завод + Область ППМ','Код округа + Материал','Склад потребности','Область ППМ','Центральный завод','Код округа','Признак аналога','Класс оценки','подгруппа','подподгруппа','Недропользование','ПлановСрокПоставки','SS (норма запаса на складе)','SS сумма','Остаток на начало года Кол-во','Остаток на начало года Сумма'],axis = 1,inplace=True).drop(list(f0.loc[:,[x for x in f0.columns if x.find('Остаток Кол-во')!=-1 and int(x.split('/')[0]) !=current_date.month ]]),axis = 1,inplace=True).drop(list(f0.loc[:,[x for x in f0.columns if x.find('Остаток Сумма')!=-1 and int(x.split('/')[0]) !=current_date.month ]]),axis = 1,inplace=True).drop([x for x in f0.columns if x.find('ФП')!=-1 or x.find('ФC')!=-1 or x.find('БЗ')!=-1 or x.find('Остаток прогноз')!=-1],axis = 1)

In [ ]:
f0 =  get_in_backup(path_backup='obesp_no_f_10.h5',name='f0_cons_10')

In [ ]:
f0

In [ ]:
spr_mat_kart_crm_inic_zak_znb = rev_to_dict(ZNB['Материал'].astype('str')+' '+ZNB ['Карточка RCM'], ZNB['Инициатор закупки']     )

In [ ]:
ZNB ['Карточка RCM'].unique()

In [ ]:
ZNB['Материал'].unique()

In [ ]:
f0['Материал+RCM'] = f0['Материал']+' '+f0['RCM'].astype('str')
f0['Трафик'] = f0['Материал+RCM'].apply(lambda x:  insert_vpr(x,spr_mat_kart_crm_inic_zak_znb,'Инициатор закупки'))
f0.drop('Материал+RCM',axis = 1,inplace = True)

In [ ]:
f0['Трафик'].unique()

In [ ]:
f0

In [ ]:
f0.loc[ind,'ПП кол-во '+str(current_date.month+1)+'/'+ str(current_date.year):'ПП кол-во '+str(current_date.month+3)+'/'+ str(current_date.year)].sum()

In [ ]:
f0

In [ ]:
current_date.month-1

In [ ]:
current_date.month+3

In [ ]:
'ПП кол-во '+str(current_date.month+1)+'/'+ str(current_date.year)

In [ ]:
'ПП кол-во '+str(current_date.month+3)+'/'+ str(current_date.year)

In [ ]:
f0['Дата заключения RCM'].unique()

In [ ]:
f0['СтатусМтрл на заводе'].unique()

In [ ]:
f0

In [ ]:
f0.rename(columns={'Остаток на начало года Кол-во':str(current_date.month)+'/'+str(current_date.year)+' Остаток Кол-во',\
                  'Остаток на начало года Сумма':str(current_date.month)+'/'+str(current_date.year)+' Остаток Сумма'},inplace=True)


In [ ]:
f0

In [ ]:
f0

In [ ]:
f0

In [ ]:
f0.shape

In [ ]:
f0['Категория ТМЦ'].unique()

In [ ]:
f0.shape

In [ ]:
tt = get_in_backup(path_backup='backup_obesp.h5',name='obesp_no_f_01')

In [ ]:
tt.shape

In [ ]:

f0 = f0[f0['Материал'].apply(lambda x:len(x)<=10)].shape

In [ ]:
current_date = datetime.datetime(2020,0,1)

In [ ]:
current_date.month

In [ ]:
current_date = 0

In [ ]:
current_date._month = 0

In [ ]:
class c_(datetime.datetime):
    pass


In [ ]:
current_date = c_

In [ ]:
current_date.month = 0

In [ ]:
datetime.datetime

In [ ]:
current_date.month = 0

In [ ]:
datetime.datetime(2019,1,1)

In [ ]:
current_date

In [ ]:
'ПП кол-во '+str(current_date.month+3)+'/'+ str(current_date.year)

In [ ]:
current_date.month

In [ ]:
current_date.year

In [ ]:
current_date = datetime.datetime(2020,1,1)

In [ ]:
current_date.month = 0

In [ ]:
current_date.year = 2020

In [ ]:
current_date

In [ ]:
current_date.year

In [ ]:
tt = exlsx_f(f0,2020,0,ppm_date,nach_goda)

In [ ]:
f0 =  get_in_backup(path_backup='backup_obesp_t.h5',name='obesp_no_f_01_t')

In [ ]:
nach_goda = True
ppm_date = datetime.datetime(2020,1,8)
if nach_goda:
    current_date = c_
    current_date.month = 0
    current_date.year = 2020

In [ ]:
current_date = datetime.datetime(2020,1,1)

In [ ]:
ppm_date = datetime.datetime(2020,1,8)

In [ ]:
nach_goda = True

In [ ]:
f0['Цена']

In [ ]:
f0 =  get_in_backup(path_backup='backup_f0.h5',name='c_f0_01_ost')

In [ ]:
f0['Цена']

In [ ]:
current_date_f0 = datetime.datetime(2020,1,1)
first_month_of_kvartal = 1
sp = ['ПП кол-во '+str(x)+'/'+str(current_date_f0.year) for x in  range(first_month_of_kvartal, current_date_f0.month+1)]

In [ ]:
f0.loc[:,'ПП кол-во '+str(month+1)+'/'+ str(year):'ПП кол-во '+str(month+3)+'/'+ str(year)]

In [ ]:
f0.loc[:,sp_k].sum(axis=1)

In [ ]:
f0

In [ ]:
f0['ПП 3 месяца']  = f0.loc[:,sp_k].sum(axis=1)
f0['ПП 3 месяца,Сумма']  = f0.loc[:,sp_summ].sum(axis=1)


f0['Обеспечено 3 месяца, кол-во'] = f0.apply(lambda x:func_obesp_3month(x,year,month,sp_k),axis = 1 )

In [ ]:
['ПП кол-во '+str(x)+'/'+str(current_date_f0.year) for x in  range(first_month_of_kvartal, current_date_f0.month+1)]

In [ ]:
current_date_f0 = datetime.datetime(2020,12,1)
first_month_of_kvartal = 10
sp_k = ['ПП кол-во '+str(x)+'/'+str(current_date_f0.year) for x in  range(first_month_of_kvartal, current_date_f0.month+1)]
sp_summ = ['ПП сумма '+str(x)+'/'+str(current_date_f0.year) for x in  range(first_month_of_kvartal, current_date_f0.month+1)]
print(sp_k)
print(sp_summ)

In [ ]:
f0['Обеспечено 3 месяца, кол-во'] = f0.apply(lambda x:func_obesp_3month(x,year,month,sp_k),axis = 1 )

In [ ]:
f0

In [ ]:
f0.loc[0][sp_kv].sum()

In [ ]:
f0

In [ ]:
f0.loc[ind,sy].values

In [ ]:
ind

In [ ]:
for ind in f0.index[:10000]:
    print([1 for x in  sy  if  f0.loc[ind,x].sum()>1])


In [ ]:
f0['Периодичность'].unique()

In [ ]:
f0[(f0['ПП кол-во 1/2020']>0)&(f0['ПП кол-во 9/2020']>0)]

In [ ]:
f0

In [ ]:
f0 =  get_in_backup(path_backup='backup_f0.h5',name='c_f0_01_ost')

In [ ]:
f0 =  get_in_backup(path_backup='backup_f0_cons.h5',name='c_f0_cons_10')

In [ ]:
f0

In [ ]:
'ПП кол-во '+str(1)+'/'+ str(year)


In [ ]:
'ПП кол-во '+str(month)+'/'+ str(year)

In [ ]:
f0.loc[:,'ПП кол-во '+str(1)+'/'+ str(year):'ПП кол-во '+str(month)+'/'+ str(year)]

In [ ]:
print(year)
print(month)

In [ ]:
f0

In [ ]:
save_pyex(df=f0,path='C:\\Users\\Илья\\Desktop\\kazzinc\\Uploads1\\extr_xlsx\\.xlsx')

In [ ]:
f0

In [ ]:
f0.rename(columns={'Остаток на начало года Кол-во':str(1)+'/'+str(year)+' Остаток Кол-во',\
                  'Остаток на начало года Сумма':str(1)+'/'+str(year)+' Остаток Сумма'},inplace=True)

In [ ]:
f0[str(1)+'/'+str(year)+' Остаток Кол-во']  = f0['Остаток на начало года Кол-во']
f0[str(1)+'/'+str(year)+' Остаток Сумма']  = f0['Остаток на начало года Сумма']

In [ ]:
sp_kv

In [ ]:
def func_obesp_second_month_procentil(x,year,month,sp):
    if x['ПП кол-во '+str(month+2)+'/'+ str(year)] ==0:
        return ''
    elif x['ПП кол-во '+str(month+1)+'/'+ str(year)]>x[str(month)+'/'+str(year)+' Остаток Кол-во']:
        return 0
    elif ((x[str(month)+'/'+str(year)+' Остаток Кол-во']-yy(x,sp))/x['ПП кол-во '+str(month+2)+'/'+ str(year)])>1:
        return 1
    else:
        return ((x[str(month)+'/'+str(year)+' Остаток Кол-во']-yy(x,sp))/x['ПП кол-во '+str(month+2)+'/'+ str(year)])

In [ ]:
f01['Обеспеченность ' + str(month+2)+'/'+str(year)+ ' , кол-во, %'] = f01.apply(lambda x:func_obesp_second_month_procentil(x,year,month,sp_kv_primech),axis = 1 )

In [ ]:
f01 = f0[:10].copy()

In [ ]:
f01

In [ ]:
last_month_of_kvartal = 3
sp_kv_sum_primech = ['ПП сумма '+str(x)+'/'+str(current_date_f0.year) for x in  range(first_month_of_kvartal, last_month_of_kvartal+1)]

In [ ]:
sp_kv_sum_primech

In [ ]:
def func_obesp_third_month_procentil(x,year,month,sp):
    if x['ПП кол-во '+str(month+3)+'/'+ str(year)] ==0:
        return ''
    elif (yy(x,sp))>x[str(month)+'/'+str(year)+' Остаток Кол-во']:
        return 0
    elif ((x[str(month)+'/'+str(year)+' Остаток Кол-во']-yy(x,sp)-x['ПП кол-во '+str(month+2)+'/'+ str(year)])/x['ПП кол-во '+str(month+3)+'/'+ str(year)])>1:
        return 1
    else:
        return ((x[str(month)+'/'+str(year)+' Остаток Кол-во']-yy(x,sp))/x['ПП кол-во '+str(month+3)+'/'+ str(year)])

In [ ]:
f01['Обеспеченность ' + str(month+2)+'/'+str(year)+ ' , кол-во, %'] = f01.apply(lambda x:func_obesp_third_month_procentil(x,year,month,sp_current_kvartal),axis = 1 )

In [ ]:
f01

In [ ]:
def func_ne_obesp_third_month_(x,year,month,sp):
    if (x['ПП кол-во '+str(month+3)+'/'+ str(year)]-x[str(month)+'/'+str(year)+' Остаток Кол-во']+yy(x,sp)-x['Не обеспечено запасом ' + str(month+1)+'/'+str(year)+ ' , кол-во']-x['Не обеспечено запасом ' + str(month+2)+'/'+str(year)+ ' , кол-во'])>0:
        return x['ПП кол-во '+str(month+3)+'/'+ str(year)]-x[str(month)+'/'+str(year)+' Остаток Кол-во']+yy(x,sp)-x['Не обеспечено запасом ' + str(month+1)+'/'+str(year)+ ' , кол-во']-x['Не обеспечено запасом ' + str(month+2)+'/'+str(year)+ ' , кол-во']
    else:
        return 0
    

In [ ]:
f01['Не обеспечено запасом ' + str(month+3)+'/'+str(year)+ ' , кол-во'] = f01.apply(lambda x:func_ne_obesp_third_month_(x,year,month,sp_current_kvartal),axis = 1 )

In [ ]:
f01

In [ ]:
sp_current_kvartal

In [ ]:
str(month)+'/'+str(year)+' Остаток Кол-во'

In [ ]:
yy(f01.loc[0],sp_current_kvartal)

In [ ]:
f01.loc[0]

In [ ]:
sp_current_kvartal

In [ ]:
f0

In [ ]:
f4 =  get_in_backup(path_backup='backup_f0_cons.h5',name='c_f0_cons_02')

In [ ]:
f4

In [ ]:
f0 =  get_in_backup(path_backup='backup_f0_cons.h5',name='c_f0_cons_02')

In [ ]:
f0.shape

In [ ]:
f0[f0['Материал']=='870208801']

In [ ]:
f0

In [ ]:
folder = '0320'

In [ ]:
f0

In [ ]:
f0 =  get_in_backup(path_backup='f0_cons_04_12032020_v2.h5',name='f0_cons_021')

In [ ]:
f0

In [ ]:
seven =  get_in_backup(path_backup='backup_f0.h5',name='c_f0_07_20_v2')

In [ ]:
seven['Категория_запаса'].unique()

In [ ]:
cons =  get_in_backup(path_backup='cons__05_with_ost.h5',name='v_2_with_ost')

In [ ]:
cons['Категория ТМЦ'].unique()

In [ ]:
cons =  get_in_backup(path_backup='cuo_cons_07.h5',name='cuo_cons_07')

In [ ]:
cons['Базовая ЕИ'].value_counts()

In [ ]:
cons[cons['Базовая ЕИ']=='']

In [ ]:
f0

In [ ]:
# строка только для начала года
if nach_goda:
    f0.rename(columns={'Остаток на начало года Кол-во':str(month)+'/'+str(year)+' Остаток Кол-во',\
                  'Остаток на начало года Сумма':str(month)+'/'+str(year)+' Остаток Сумма'},inplace=True)

f0.reset_index(drop=True,inplace=True)
f0 = f0[(f0['Категория ТМЦ']=='Текущая потребность')&(f0.loc[:,[x for x in f0.columns if x.find('ПП кол-во')!=-1]].sum(axis = 1)>0)]
f0['СтатусМтрл на заводе'] = f0['СтатусМтрл на заводе'].apply(lambda x: 0 if x=='' else x).astype('int').astype('str')
f0['СтатусМтрл на заводе'] = f0['СтатусМтрл на заводе'].apply(lambda x: str(x).replace('.0','')).astype('int').astype('str')
if nach_goda:
    f0 = f0.drop(['Категория ТМЦ','БЕ','Имя Завода + Материал + Заказ','Завод + Склад','Завод + Область ППМ',\
                  'Код округа + Материал','Склад потребности','Область ППМ','Центральный завод','Код округа','Признак аналога',\
                  'Класс оценки','подгруппа','подподгруппа','Недропользование','ПлановСрокПоставки','SS (норма запаса на складе)'\
                  ,'SS сумма'],axis = 1).drop(list(f0.loc[:,[x for x in f0.columns if x.find('Остаток Кол-во')!=-1 and int(x.split('/')[0]) !=month ]]),axis = 1).drop(list(f0.loc[:,[x for x in f0.columns if x.find('Остаток Сумма')!=-1 and int(x.split('/')[0]) !=month ]]),axis = 1).drop([x for x in f0.columns if x.find('ФП')!=-1 or x.find('ФC')!=-1 or x.find('БЗ')!=-1 or x.find('Остаток прогноз')!=-1],axis = 1)
    
else:
        
    f0 = f0.drop(['Категория ТМЦ','БЕ','Склад потребности','Область ППМ','Центральный завод','Код округа','Признак аналога','Класс оценки','подгруппа','подподгруппа','Недропользование','ПлановСрокПоставки','SS (норма запаса на складе)','SS сумма','Кол-во запаса в начале периода','Сумма запаса в начале периода'],axis = 1).drop(list(f0.loc[:,[x for x in f0.columns if x.find('Остаток Кол-во')!=-1 and int(x.split('/')[0]) !=month ]]),axis = 1).drop(list(f0.loc[:,[x for x in f0.columns if x.find('Остаток Сумма')!=-1 and int(x.split('/')[0]) !=month ]]),axis = 1).drop([x for x in f0.columns if x.find('ФП')!=-1 or x.find('ФC')!=-1 or x.find('БЗ')!=-1 or x.find('Остаток прогноз')!=-1],axis = 1)

f0.loc[f0[f0['Заказ'].str.startswith('2')].index,'Заказ'] = '0'
aggregate= dict((x,'sum') for x in f0.columns if f0[x].dtype=='float64' and x!='Цена')
aggregate['Цена'] = ['max']
f0 = f0.groupby([x for x in f0.columns if f0[x].dtype=='object']).agg(aggregate).reset_index()
f0.columns =  [x[0] for x in f0.columns]
# f0[[x for x in f0.columns if f0[x].dtype=='float64']] = f0[[x for x in f0.columns if f0[x].dtype=='float64']].apply( lambda x: round(x,3))

In [ ]:
a= list(f0.loc[:,:'РР'].columns)

In [ ]:
a.extend(['Цена'])

In [ ]:
a

In [ ]:
f0.groupby(a).sum().reset_index()

In [ ]:
f0['СтатусМтрл на заводе'] = f0['СтатусМтрл на заводе'].astype('int')

f0['Заказ+Материал'] = f0['Заказ']+' '+f0['Материал']
iw13['Заказ'] = iw13['Заказ'].astype('str')
spr_zak_mat_iw13 = rev_to_dict(iw13['Заказ']+' '+iw13['Материал'],iw13 ['Дата деблокирования'])
f0['Дата деблокирования (ТОРО)'] = f0['Заказ+Материал'].apply(lambda x:  insert_vpr(x,spr_zak_mat_iw13,'Дата деблокирования'))
f0.drop('Заказ+Материал',axis = 1,inplace = True)

f0['Завод+НН'] = f0['Завод'].astype('str')+' #'+f0['Материал'].astype('str')

spr_data_sozdanya_potr_df = rev_to_dict(df['Завод+НН'],df ['Дата создания потребности'])
f0['Дата создания потребности'] = f0['Завод+НН'].apply(lambda x:  insert_vpr(x,spr_data_sozdanya_potr_df,'Дата создания потребности'))
f0['Дата создания потребности'] = f0['Дата создания потребности'].apply(lambda x: '01.01.1900' if x =='00.00.0000' else  x)
f0['Дата создания потребности'] = f0['Дата создания потребности'].apply(lambda x: '01.01.1900' if x ==0  else  x)
f0['Дата создания потребности'] = [datetime.datetime(int(x.split('.')[2]),int(x.split('.')[1]),int(x.split('.')[0])) for x in f0['Дата создания потребности'].values]

spr_data_postaki_dlya_el_popolnreniya_ds = rev_to_dict(df['Завод+НН'],df ['Дата поставки для элемента пополнения'])
f0['Дата поставки для элемента пополнения'] = f0['Завод+НН'].apply(lambda x:  insert_vpr(x,spr_data_postaki_dlya_el_popolnreniya_ds,'Дата поставки для элемента пополнения'))
f0['Дата поставки для элемента пополнения'] = f0['Дата поставки для элемента пополнения'].apply(lambda x: '01.01.1900' if x =='00.00.0000' else  x)
f0['Дата поставки для элемента пополнения'] = f0['Дата поставки для элемента пополнения'].apply(lambda x: '01.01.1900' if x ==0  else  x)
f0['Дата поставки для элемента пополнения'] = [datetime.datetime(int(x.split('.')[2]),int(x.split('.')[1]),int(x.split('.')[0])) for x in f0['Дата поставки для элемента пополнения'].values]

f0['Заказчик + HH'] =f0['Комплекс'].astype('str')+' '+f0['Материал'].astype('str')


ZNB = pd.read_excel(directory+'ZNB.xlsx',sheet_name='Sheet1')
UB05 = pd.read_excel(directory+'UB05.xlsx',sheet_name='Sheet1')
spr_znb = pd.read_excel(directory+'Справодник для ZNB.xlsx',sheet_name='ZNB')
ZNB.fillna(0,inplace=True)
ZNB['Склад'] = ZNB['Склад'].apply(lambda x: '' if x==0 else x)
ZNB['Завод'] = ZNB['Завод'].apply(lambda x: str(x).replace('.0','')).astype('str')
ZNB['Завод+Склад'] = ZNB['Завод'].astype('str')+' '+ZNB['Склад'].astype('str')
spr_znb_ = rev_to_dict(spr_znb['Завод+склад'],spr_znb['Комплекс'])
ZNB['Комплекс']=ZNB['Завод+Склад'].apply(lambda x:  insert_vpr(x,spr_znb_,'Комплекс'))
ZNB['Комплекс+Маттериал'] = ZNB['Комплекс'].astype('str')+' '+ZNB['Материал'].astype('str')
ZNB['Завод+НН'] = ZNB['Завод'].astype('str')+' #'+ZNB['Материал'].astype('str')
ZNB.fillna(0,inplace=True)
ZNB['Карточка RCM'] = ZNB['Карточка RCM'].apply(lambda x: str(x).replace('.0','')).astype('str')




spr_kompl_mat_komp_znb_full = rev_to_dict(ZNB['Комплекс+Маттериал'],ZNB ['Карточка RCM'])
spr_mat_znb = rev_to_dict(ZNB['Материал'],ZNB ['Карточка RCM'])

for ind in f0.index:
    if (f0.at[ind,'Товар в пути (ZNB5, ZNB2/3-АвизОт)']   + f0.at[ind,'Законтрактовано(ZNB-ПртПЗк)'])>0:
        if insert_vpr(f0.at[ind,'Заказчик + HH'],spr_kompl_mat_komp_znb_full,'Карточка RCM')==0 and f0.at[ind,'Комплекс'] in  filter_else['one'].values:
            f0.at[ind,'RCM']  = insert_vpr(f0.at[ind,'Материал'],spr_mat_znb,'Карточка RCM')
        else:
            f0.at[ind,'RCM'] = insert_vpr(f0.at[ind,'Заказчик + HH'],spr_kompl_mat_komp_znb_full,'Карточка RCM')
    

spr_mat_kart_crm_inic_zak_znb = rev_to_dict(ZNB['Материал'].astype('str')+' '+ZNB ['Карточка RCM'], ZNB['Инициатор закупки']     )
f0['Материал+RCM'] = f0['Материал']+' '+f0['RCM'].astype('str')
f0['Трафик'] = f0['Материал+RCM'].apply(lambda x:  insert_vpr(x,spr_mat_kart_crm_inic_zak_znb,'Инициатор закупки'))
f0.drop('Материал+RCM',axis = 1,inplace = True)


rcm['Ид. случая'] = rcm['Ид. случая'].astype('str')
spr_dat_zak = rev_to_dict(rcm['Ид. случая'], rcm['Дата заключения'])
spr_dat_rcm_dat_okonch = rev_to_dict(rcm['Ид. случая'], rcm['Дата окончания действия'])
spr_dat_rcm_status = rev_to_dict(rcm['Ид. случая'], rcm['Статус названия'])
f0['Дата заключения RCM'] = f0['RCM'].apply(lambda x:  insert_vpr(x,spr_dat_zak,'Дата заключения'))
f0['Дата окончания RCM'] = f0['RCM'].apply(lambda x:  insert_vpr(x,spr_dat_rcm_dat_okonch,'Дата окончания действия'))
f0['Статус договора'] = f0['RCM'].apply(lambda x:  insert_vpr(x,spr_dat_rcm_status,'Статус названия'))

f0['Дата окончания RCM'].fillna(datetime.datetime(1900,1,1),inplace = True)
f0['Дата окончания RCM'] = f0['Дата окончания RCM'].apply(lambda x: datetime.datetime(1900,1,1) if x ==0  else x)
f0['Дата окончания RCM'] = f0['Дата окончания RCM'].apply(lambda x: 1900 if  x.year == 9999 else x)

f0['Дата заключения RCM'].fillna(datetime.datetime(1900,1,1),inplace = True)
f0['Дата заключения RCM'] = f0['Дата заключения RCM'].apply(lambda x: datetime.datetime(1900,1,1) if x ==0  else x)
f0['Дата заключения RCM'] = f0['Дата заключения RCM'].apply(lambda x: 1900 if  x.year == 9999 else x)

f0['Дата деблокирования (ТОРО)'].fillna(datetime.datetime(1900,1,1),inplace = True)
f0['Дата деблокирования (ТОРО)'] = f0['Дата деблокирования (ТОРО)'].apply(lambda x: datetime.datetime(1900,1,1) if x ==0  else x)
f0['Дата деблокирования (ТОРО)'] = f0['Дата деблокирования (ТОРО)'].apply(lambda x: 1900 if  x.year == 9999 else x)





NB = pd.read_excel(directory+'NB.xlsx',sheet_name='Sheet1')
NB = NB[~NB['Обозначение индикатора деблокирования'].isin(['Тендер завершен','блокиров.'])]
spr_nb = pd.read_excel(directory+'Справодник для NB.xlsx',sheet_name='NB')
NB.fillna(0,inplace = True)
spr_nb.fillna(0,inplace = True)
NB['Количество пополнения'] = NB['ЗатребованКолич']-NB['Заказанное к-во']
NB['Завод'] = NB['Завод'].apply(lambda x: str(x).replace('.0','')).astype('str')
NB['Склад'] = NB['Склад'].apply(lambda x: str(x).replace('.0','')).astype('str')
NB['Материал'] = NB['Материал'].apply(lambda x: str(x).replace('.0','')).astype('str')
NB['Материал'] = NB['Материал'].apply(lambda x: '0'+str(x) if  len(str(x))<9 else x)
NB['Склад'] = NB['Склад'].apply(lambda x: '' if x=='0' else x)
NB['Завод+Склад'] = NB['Завод']+' '+NB['Склад']
NB['Завод+Склад'] = NB['Завод+Склад'].str.strip()
spr_nb1 = rev_to_dict(spr_nb['Завод + Склад'].str.strip(),spr_nb['Комплекс'])
NB['Комплекс'] = NB['Завод+Склад'].apply(lambda x: insert_vpr(x,spr_nb1,'Комплекс'))
NB['Комплекс'] = NB['Комплекс'].apply(lambda x: str(x).replace('.0','')).astype('str')
NB['Комплекс+Маттериал'] = NB['Комплекс']+' '+NB['Материал']
NB['Количество пополнения'] = NB['Количество пополнения'].apply(lambda x:0 if x<0 else x)


spr_kompl_mat_komp_dat_zay = rev_to_dict(NB['Комплекс+Маттериал'],NB ['Дата заявки'])
spr_mat_dat_zay = rev_to_dict(NB['Материал'],NB ['Дата заявки'])

for ind in f0.index:
    if f0.at[ind,'Заявка на закупку(NB)']>0:
        if insert_vpr(f0.at[ind,'Заказчик + HH'],spr_kompl_mat_komp_dat_zay,'Дата заявки')==0 and f0.at[ind,'Комплекс'] in  filter_else['one'].values:
            if insert_vpr(f0.at[ind,'Материал'],spr_mat_dat_zay,'Дата заявки')==0:
                f0.at[ind,'Дата создания элемента ППМ пополнения NB01'] = datetime.datetime(1900,1,1)
            else:
                f0.at[ind,'Дата создания элемента ППМ пополнения NB01']  = insert_vpr(f0.at[ind,'Материал'],spr_mat_dat_zay,'Дата заявки')
        else:
            if insert_vpr(f0.at[ind,'Заказчик + HH'],spr_kompl_mat_komp_dat_zay,'Дата заявки')==0:
                f0.at[ind,'Дата создания элемента ППМ пополнения NB01'] = datetime.datetime(1900,1,1)
            else:
                f0.at[ind,'Дата создания элемента ППМ пополнения NB01'] = insert_vpr(f0.at[ind,'Заказчик + HH'],spr_kompl_mat_komp_dat_zay,'Дата заявки')
                
                

                
                
                
f0.fillna(0,inplace=True)  

In [ ]:
f0

In [ ]:
f0[f0['Заказ']==0]

In [ ]:
f0

In [ ]:
seven =  get_in_backup(path_backup='backup_f0.h5',name='c_f0_10_20_v2')

In [ ]:
tt  = get_in_backup(path_backup='backup_for_before_transp1.h5',name='f0_vertical_10_20_raspr')

In [ ]:
seven['ПП кол-во 13/2020'].sum()

In [ ]:
tt['month'].dt.year

In [ ]:
tt[(tt['month'].dt.year==2021)&(tt['month'].dt.month==1)]['Количество потребности'].sum()

In [ ]:
import dask.set_options

In [ ]:
c_f0 =  get_in_backup(path_backup='backup_f0_.h5',name='c_f0_12_20_v2')

In [ ]:
f0[(f0['Материал']=='050162002')&(f0['Комплекс']=='ЖГОК')&(f0['Завод']=='3103')]

In [ ]:
cons =  get_in_backup(path_backup='cons__11_with_ost.h5',name='v_2_with_ost')

In [ ]:
cons

In [ ]:
cons =  get_in_backup(path_backup='cons__11_with_ost.h5',name='v_2_with_ost')

In [ ]:
cons

In [ ]:
c_f0 =  get_in_backup(path_backup='backup_f0_.h5',name='c_f0_12_20_v2')